##### Ambiente de Apoyo al Aprendizaje de Estructuras de Datos en Python
**Desarrollado por:** Vhanessa Cardona Cañaveral (201728751)

Contenido del Notebook:
Este Notebook se encuentra estructurado por secciones, a continuación se presenta el orden, y al inicio de cada sección se describe brevemente el contenido de la misma.
1. Inicialización
    * Librerías
    * Inicialización de la Interfaz
    * Métodos de Soporte
2. Metodos Gráficos (Estructuras de datos)
    * Métodos Gráficos - Listas Enlazadas
    * Métodos Gráficos - Árboles BST
    * Métodos Gráficos - Grafos
3. Validación de Operaciones
    * Validación - Listas Enlazadas
    * Validación - Árboles BST
    * Validación - Grafos
4. Componente de Enlace
    * Enlace - Listas Enlazadas
    * Enlace - Árboles BST 
    * Enlace - Grafos
5. Interfaz Grafica
    * Opciones
        * Opciones - Listas Enlazadas
        * Opciones - Árboles BST 
        * Opciones - Grafos
    * Layouts
        * Layout - Listas Enlazadas
        * Layout - Árboles BST 
        * Layout - Grafos
    * Contruir Interfaz
        * Contruir Layouts
        * Contruir Opciones
        * Contruir Aplicación
6. Mostrar Aplicación

## 1. Inicializacion

### Librerías

Se importan todas las librerías que se requieren para ejecutar el Notebook. Es importante contar con todas ellas para poder acceder a las funcionalidades de la aplicación

In [1]:
import os
os.environ["PATH"] += os.pathsep + 'C:/Program Files/Graphviz/bin/'

# Estructuras de datos que se toman como referencia para validaciones
from referencias.lista_disc import listaEnlazada as referenciaLista
from referencias.arbol_disc import bst as referenciaArbol
from referencias.grafo_disc import grafo as referenciaGrafo

# Librerias que permiten crear la interfaz grafica de la aplicacion
import ipywidgets as widgets
from ipywidgets import *
from IPython.display import display, clear_output, SVG, HTML

# Libreria que permite representar graficamente las estructuras de datos trabajadas
from graphviz import Digraph, Graph

# Librerias adicionales para procesos que se requieren durante la ejecución
import matplotlib.pyplot as plt
import random
import string
import json
import  tkinter as tk
from tkinter import filedialog
import shutil
import importlib.util
import functools

### Inicializacion de la Interfaz

Se crean los elementos base que permiten manejar una interfaz grafica. Se crea en forma de Tabs para diferenciar entre estructuras de datos

In [2]:
out, out1, out2, out3 = Output(), Output(), Output(), Output()
tab = Tab(children = [out1, out2, out3],
          layout=Layout(width='99%', height='auto'))
tab.set_title(0, 'Listas Enlazadas')
tab.set_title(1, 'BST')
tab.set_title(2, 'Grafos')

form_item_layout = Layout(
    justify_content='center'
)

### Metodos de soporte

Se definen una serie de funciones que se usan durante la ejecución, y se inicializan algunas variables

In [3]:
VALIDATION_STATES = {0:'WARNING', 1:'SUCCESSFUL', -1:'FAILED'} # Estados resultantes del proceso de validacion
colorPointer = 'grey'   # Color para apuntadores
colorRight = 'blue'     # Color para conexiones a la derecha
colorLeft = 'green'     # Color para conexiones a la izquierda
colorHigh = 'red'       # Color para resaltar un elemento
col = "black"           # Color por defecto

def checkAlgoritmGraph(type, recorrido):
    '''
    Verifica si un algoritmo de grafos se puede aplicar sobre el grafo actual 

    Args:
        type: tipo del grafo actual: 4 (dirigido) 5(no dirigido)
        recorrido: Nombre del recorrido/algoritmo que se va a ejecutar
    
    Returns:
        True si el recorrido se puede aplicar al tipo de grafo actual.
        False de lo contrario
    '''
    both = ['Bellman-Ford', 'DepthFirstSearch', 'BreadhtFirstSearch', 'DepthFirstOrder', 'Dijkstra']
    result = False
    comment = ''
    if recorrido in both:
        result = True
    elif recorrido == 'PrimMST':
        result = type == 5
        comment = 'ERROR: ' + recorrido + ' solo se puede aplicar a Grafos No Dirigidos'
    elif recorrido == 'KosarajuSCC' or recorrido == 'DirectedCycle':
        result = type == 4
        comment = 'ERROR: ' + recorrido + ' solo se puede aplicar a Grafos Dirigidos'
    return result, comment   

def getNodesGivenEdges(edges):
    '''
    Dado una lista de conexiones, retorna una lista con los nodos que se encuentran en las conexiones

    Args:
        edges: Lista de conexiones. 
        Cada conexión es una tupla: (nodo_origen, nodo_destino, peso)
    
    Returns:
        Lista de los valores de los nodos que se encuentran en las conexiones
    '''
    nodes = list()
    for i in edges:
        if i[0] not in nodes:
            nodes.append(i[0])
        if i[1] not in nodes:
            nodes.append(i[1])
    return nodes

def defaultfunction(elem_1, elem_2):
    '''
    Función de comparación entre dos elementos. Si son del mismo tipo de dato se comparan directamente, 
    de lo contrario se transforman ambos elementos a String y se realiza la comparación
    
    Args:
        elem_1: Elemento 1. 
        elem_2: Elemento 2.
    
    Returns:
        -1: Si el elemento 1 es menor al elemento 2
         0: Si el elemento 1 es igual al elemento 2
         1: Si el elemento 1 es mayor al elemento 2
    '''
    if type(elem_1) != type(elem_2):
        elem_1 = str(elem_1)
        elem_2 = str(elem_2)
    if elem_1 > elem_2:
        return 1
    elif elem_1 < elem_2:
        return -1
    return 0

def create_n_random(n):
    '''
    Crea una lista de numeros aleatorios sin repetición de longitud n.
       
    Args:
        n: tamaño de la lista
    
    Returns:
        Lista de n numeros aleatorios sin repeticion
    '''
    nodos = list()
    for i in range(n*n):
        num = random.randint(1,50)
        if num not in nodos:
            nodos.append(num)
        if len(nodos) == n:
            break
    return nodos 

def createTuples(nodos, tipo):
    '''
    Crea una lista de tuplas (id, elemento) creando las instancias adicionales (apuntadores)
    que se tienen en una lista enlazada dependiendo del tipo
       
    Args:
        nodos: elementos de la lista enlazada
        tipo: tipo de lista enlazada. (1: sencilla, 2: doble)
    
    Returns:
        Lista de tuplas de los elementos de nodos con ids unicos
    '''
    tuples = list()
    if tipo == 1:
        ids = range(0,len(nodos)+2)
        tuples.append((ids[0],'First'))
        for i in range(1,len(ids)-1):
            tuples.append((ids[i],nodos[i-1]))
        if len(nodos) > 0:
            tuples.append((ids[len(nodos)+1],'None'))
    else:
        ids = range(0,len(nodos)+3)
        tuples.append((ids[0],'First'))
        if len(nodos) > 0:
            tuples.append((ids[1],'None'))
            for i in range(2,len(ids)-1):
                tuples.append((ids[i],nodos[i-2]))
            tuples.append((ids[len(nodos)+2],'None'))
    return tuples

def get_random_string():
    '''
    Crea una String random de tamaño 20
       
    Args:
    
    Returns:
        String random de 20 caracteres
    '''
    letters = string.ascii_lowercase
    result_str = ''.join(random.choice(letters) for i in range(20))
    return result_str

def create_n_randomEdges(nodos, tipo = 4):
    '''
    Dado una lista de valores de nodos, se crean x conexiones entre ellos, donde
    n es un numero aleatorio entre 10 y la cantidad de nodos. 
       
    Args:
        nodos: lista de los elementos de los nodos de un grafo
        tipo: tipo de grafo (4: Dirigido, 5: No Dirigido)
    
    Returns:
        Lista de tuplas (nodo_origen, nodo_destino, costo)
    '''
    edges = list()
    edgesAux = list()
    ln = random.randint(10, len(nodos))
    for i in range(ln):
        a = random.choice(nodos)
        b = random.choice(nodos)
        while a == b:
            b = random.choice(nodos)
        if (a,b) not in edgesAux:
            if tipo != 4:
                if (b,a) not in edgesAux:
                    num = random.random() + random.randint(1,50)
                    edges.append((a,b,num))
                    edgesAux.append((a,b))
            else:
                num = random.random() + random.randint(1,50)
                edges.append((a,b,num))
                edgesAux.append((a,b))
    return edges

def validarEstructura(valid, tipo):
    '''
    Verifica si el tipo de una estructura de datos se enuentra entre las validas (valid) 
       
    Args:
        valid: lista de tipos de estructuras validas
        tipo: tipo de una estructura
    
    Returns:
        state: True si el tipo se encuentra en la lista de tipos validos, False de los contrario
        comment: Mensaje de error informando cual es la estructura del tipo ingresado
    '''
    if tipo in valid:
        state = True
        comment = ''
    else:
        state = False
        if tipo == 1:
            actual = 'Lista Encadenada Sencilla'
        elif tipo == 2:
            actual = 'Lista Encadenada Doble'
        elif tipo == 3:
            actual = 'Arbol BST'
        elif tipo == 4:
            actual = 'Grafo Dirigido'
        else:
            actual = 'Grafo NO Dirigido'
        comment = 'ERROR: La estructura actual es ' + actual
    return state, comment


## 2. Metodos Gráficos (Estructuras de datos)   

### Métodos Gráficos - Listas Enlazadas

In [4]:
def displayList(estructura, tipo, nodosX=list()):
    '''
    Grafica en el Canvas la lista que entra por parametro, del tipo dado, y resalta los nodos
    que se encuentren en la lista nodosX
       
    Args:
        estructura: estructura de datos que debe ser una lista enlazada
        tipo: tipo de la estructura (1: Sencilla, 2: doble)
        nodosX: lista de valores de nodos que se resaltan (pintan en otro color)
    
    Returns:
        -
    '''
    try:
        nodos = estructura.getNodeValues()
    except:
        e = '\tProblema en el método getNodeValues()'
        raise Exception(e)
    
    dot = Digraph()
    
    dot.graph_attr = {
        'rankdir': 'LR',
        'center': 'true',
        'size':'14,5',
        'ratio':'fill'
    }
    
    tuples = createTuples(nodos, tipo)
    
    for i,j in tuples:
        if j == 'First' or j == 'None':
            dot.node(name=str(i), label=str(j), shape="square", color='white')
        elif j in nodosX:
            dot.node(name=str(i), label=str(j), shape="square", color=colorHigh)
        else:
            dot.node(name=str(i), label=str(j), shape="square")
    if len(nodos) == 0:
        dot.node(name='-1', label='', shape="square", color='white')
        dot.edge(str(tuples[0][0]), str('-1'), color=colorPointer)
    else:
        for i in range(1,len(tuples)):
            if tuples[i-1][1] == 'First' and tuples[i][1] != 'None':
                dot.edge(str(tuples[i-1][0]), str(tuples[i][0]), color=colorPointer)
            elif tuples[i-1][1] == 'First' and tuples[i][1] == 'None':
                dot.edge(str(tuples[i-1][0]), str(tuples[i+1][0]), color=colorPointer)
            elif tuples[i-1][1] != 'None':
                dot.edge(str(tuples[i-1][0]), str(tuples[i][0]), color=colorRight)
        if tipo == 2:
            for i in range(1, len(tuples)-1):
                if tuples[i][1] != 'None':
                    dot.edge(str(tuples[i][0]), str(tuples[i-1][0]), color = colorLeft)
    
    if tipo == 2:
        dot2 = Digraph()
        dot2.graph_attr = {
            'rankdir': 'DT'
        }
        dot2.node(name='Colores', label='', color = 'black', shape="none", height="0.2", width = '0.01', fontsize='8.0')
        dot2.node(name='indicativoLeft', label='', color = 'black', shape="none", height="0.2", width = '0.01', fontsize='8.0')
        dot2.edge('indicativoLeft', 'Colores', color=colorLeft, minlen = '0.01', label='before', fontsize='8.0')

        dot3 = Digraph()
        dot3.node(name='Colores2', label='', color = 'black', shape="none", height="0.01", width = '0.01', fontsize='8.0')
        dot3.node(name='indicativoRight', label='', color = 'black', shape="none", height="0.01", width = '0.01', fontsize='8.0')
        dot3.edge('indicativoRight', 'Colores2', color=colorRight, minlen = '0.01', label='next', fontsize='8.0')

        dot2.subgraph(dot3)
        display(dot2)
        display(dot)
    else:
        display(dot)


### Métodos Gráficos - Árboles BST

In [5]:
from DISClib.ADT import orderedmap as omap
from DISClib.ADT import list as lt
from DISClib.Algorithms.Trees import traversal as tr

class BST_grafico():
    '''
    Clase auxiliar para graficar un arbol BST, usando la libreria graphviz
    adaptada de https://www.evamariakiss.de/apps/bstlearner_v1.php
    '''
    def __init__(self):
        self.estructura = omap.newMap(omaptype = 'BST', comparefunction = defaultfunction)

    def addNode_byValue(self, infoNodo):    
        self.estructura = omap.put(self.estructura, infoNodo, infoNodo)

    def getNodeValues(self):
        lst = list()
        if not omap.isEmpty(self.estructura):
            iter = lt.iterator(tr.preorder(self.estructura))
            for i in iter:
                lst.append(i)
        return lst

    def visualize(self, lst = []):
        tree = self.getNodeValues()
        if len(tree) > 0:
            tree = omap.get(self.estructura, tree[0])
            # Recursively add nodes and edges
            def add_nodes_edges(tree, dot=None):
                col = "black"
                # Create Graphviz Digraph 
                if dot is None:
                    dot = Digraph()
                    dot.graph_attr = {
                        'rankdir': 'TB',
                        'center': 'true',
                        'size':'14,5',
                        'ratio':'auto'
                    }
                    dot.node(name='root', label='root', color = 'white', shape="circle", fixedsize="True", width="0.4")
                    if (lst != [] and tree['value'] in lst):
                        dot.node(name=str(tree['value']), label=str(tree['value']), color = colorHigh, shape="circle", fixedsize="True", width="0.4")
                    else:
                        dot.node(name=str(tree['value']), label=str(tree['value']), color = col, shape="circle", fixedsize="True", width="0.4")
                    dot.edge('root', str(tree['value']), color=colorPointer)      
                
                # Add nodes recursively
                if tree['left'] != None:
                    if (lst != [] and tree['left']['value'] in lst):
                        col = "red"  
                    dot.node(name=str(tree['left']['value']), label=str(tree['left']['value']),
                            color = col, shape="circle", fixedsize="True", width="0.4")
                    col = "black"
                    dot.edge(str(tree['value']), str(tree['left']['value']),color=colorLeft)
                    dot = add_nodes_edges(tree['left'], dot=dot)
                else:
                    aux = get_random_string()
                    dot.node(name=aux, label='',
                            color = 'white', shape="circle", fixedsize="True", width="0.4")
                    dot.edge(str(tree['value']), aux,color=colorLeft)
                    #dot = add_nodes_edges(tree['left'], dot=dot)
                                
                if tree['right'] != None:
                    if (lst != [] and tree['right']['value'] in lst): 
                        col = "red" 
                    dot.node(name=str(tree['right']['value']), label=str(tree['right']['value']), 
                            color = col, shape="circle", fixedsize="True", width="0.4")
                    col = "black"
                    dot.edge(str(tree['value']), str(tree['right']['value']), color=colorRight)
                    dot = add_nodes_edges(tree['right'], dot=dot)            
                else:
                    aux = get_random_string()
                    dot.node(name=aux, label='',
                    color = 'white', shape="circle", fixedsize="True", width="0.4")
                    dot.edge(str(tree['value']), aux,color=colorRight)
                    #dot = add_nodes_edges(tree['left'], dot=dot)
                return dot        
            return add_nodes_edges(tree)                   

        else:
            dot = Digraph()
            dot.node(name='root', label='root', color = 'white', shape="circle", fixedsize="True", width="0.4")
            dot.node(name='-1', label='', shape="square", color='white')
            dot.edge('root', str(-1), color=colorPointer)
            return dot


In [6]:
def displayBST(estructura, nodosX = []):
    '''
    Grafica en el Canvas el arbol BST que entra por parametro y resalta los nodos
    que se encuentren en la lista nodosX
       
    Args:
        estructura: estructura de datos que debe ser una lista enlazada
        nodosX: lista de valores de nodos que se resaltan (pintan en otro color)
    
    Returns:
        -
    '''
    try:
        nodos = estructura.getNodeValues('Preorder')
    except:
        e = '\tProblema en el método getNodeValues()'
        raise Exception(e)
    
    bst = BST_grafico()
    for i in nodos:
        bst.addNode_byValue(i)
    dot = bst.visualize(nodosX)
    
    dot2 = Digraph()
    dot2.graph_attr = {
        'rankdir': 'DT'
    }
    dot2.node(name='Colores', label='', color = 'black', shape="none", height="0.2", width = '0.01', fontsize='8.0')
    dot2.node(name='indicativoLeft', label='', color = 'black', shape="none", height="0.2", width = '0.01', fontsize='8.0')
    dot2.edge('indicativoLeft', 'Colores', color=colorLeft, minlen = '0.01', label='left', fontsize='8.0')

    dot3 = Digraph()
    dot3.node(name='Colores2', label='', color = 'black', shape="none", height="0.01", width = '0.01', fontsize='8.0')
    dot3.node(name='indicativoRight', label='', color = 'black', shape="none", height="0.01", width = '0.01', fontsize='8.0')
    dot3.edge('indicativoRight', 'Colores2', color=colorRight, minlen = '0.01', label='right', fontsize='8.0')

    dot2.subgraph(dot3)
    display(dot2)
    display(dot)


### Métodos Gráficos - Grafos

In [7]:
def displayGraph(estructura, tipo, label, nodosX=list(), edgesX=list(), nodeY=None):
    '''
    Grafica en el Canvas el grafo que entra por parametro, del tipo dado, y resalta los nodos
    que se encuentren en la lista nodosX, resalta los arcos que se encuentren en la lista de edgesX,
    y si hay un valor para nodeY lo resalta de un color diferente
       
    Args:
        estructura: estructura de datos que debe ser un grafo
        tipo: tipo de la estructura (4: Dirigido, 2: No Dirigido)
        label: indica si se muestran o no los pesos de los arcos
        nodosX: lista de valores de nodos que se resaltan (pintan en otro color)
        edgesX: lista de valores de arcos (tuplas) que se resaltan (pintan en otro color)
        nodeY: nodo que se resalta en un color diferente a los anteriores
    
    Returns:
        -
    '''
    nodes = estructura.getNodeValues()
    edges = estructura.getEdgeValues()
    edgesAux = estructura.getEdgeValues()
    for i,j,k in edges:
        if j not in nodes:
            edgesAux.remove((i,j,k))    
    edges = edgesAux
    if tipo == 4:
        dot = Digraph()
    else:
        dot = Graph()
    
    dot.graph_attr = {
        'rankdir': 'TB',
        'center': 'true',
        'size':'14,5',
        'ratio':'auto',
        'layout': 'neato',
        'mode': 'sgd',
    }
    
    for i in nodes:
        if i in nodosX:
            dot.node(name=i, label=i, shape='circle',color='red')
        elif i == nodeY:
            dot.node(name=i, label=i, shape='circle',color='blue')
        else:
            dot.node(name=i, label=i, shape='circle',color='black')
    if tipo == 4:
        for i,j,k in edges:
            if (i,j) in edgesX:
                if label:
                    dot.edge(i, j, label=str(round(k,2)), color='red', fontsize='8.0')
                else:
                    dot.edge(i, j, color='red')
            else:
                if label:
                    dot.edge(i, j, label=str(round(k,2)), color='black', fontsize='8.0')
                else:
                    dot.edge(i, j, color='black')
    else:
        tuples = list()
        for i,j,k in edges:
            inv = (j, i)
            if inv not in tuples:
                if (i,j) in edgesX or inv in edgesX:
                    if label:
                        dot.edge(i, j, label=str(round(k,2)), color='red', fontsize='8.0')
                    else:
                        dot.edge(i, j, color='red')
                else:
                    if label:
                        dot.edge(i, j, label=str(round(k,2)), color='black', fontsize='8.0')
                    else:
                        dot.edge(i, j, color='black')
            tuples.append((i, j))
    display(dot)


## 3. Validación de Operaciones

En esta sección se implementan métodos que ejecutan en las estructuras de datos referencia las mismas operaciones que se realizan en la estructura de datos de prueba y determinan si el resultados de las estructuras de datos de prueba fue valido.

### Validacion - Listas Enlazadas

In [8]:
def validar_lista_crear(nodos, st_nodos, tipo):
    '''
    Valida la operacion de crear una lista enlazada
       
    Args:
        nodos: nodos que se añadieron al crear la lista enlazada
        st_nodos: nodos que la estructura tiene
        tipo: tipo de lista enlazada (1: sencilla, 2: doble)
               
    Returns:
        state_val: Estado final de la validacion (WARNING, SUCCESSFUL, FAILED)
        comment: mensaje informativo para el usuario
        
    '''
    txt_nodos = ''
    for i in nodos:
        txt_nodos = txt_nodos + str(i) + ', '
    
    txt_st = ''
    for i in st_nodos:
        txt_st = txt_st + str(i) + ', '
        
    if len(nodos) == len(st_nodos):
        if nodos == st_nodos:
            if len(nodos) != 0:
                comment = 'Nodos: ' + txt_nodos[:-2]
            else:
                comment = 'Nodos: []'
            state_val = VALIDATION_STATES[1]
        else:
            nodos = sorted(nodos, key=functools.cmp_to_key(defaultfunction))
            st_nodos = sorted(st_nodos, key=functools.cmp_to_key(defaultfunction))
            
            if nodos == st_nodos:
                comment = 'Se añadieron los elementos pero en un orden diferente'
                comment = comment + '\nSe esperaba: ' + txt_nodos[:-2]
                comment = comment + '\n  Se obtuvo: ' + txt_st[:-2]
                state_val = VALIDATION_STATES[0]
            else:
                comment = 'Se añadieron elementos diferentes a los indicados'
                comment = comment + '\nSe esperaba: ' + txt_nodos[:-2]
                comment = comment + '\n  Se obtuvo: ' + txt_st[:-2]
                state_val = VALIDATION_STATES[-1]
    else:
        comment = 'Se añadieron más elementos de los esperados'
        comment = comment + '\nSe esperaba: ' + txt_nodos[:-2]
        comment = comment + '\n  Se obtuvo: ' + txt_st[:-2]
        state_val = VALIDATION_STATES[-1]
        
    return state_val, comment

def validar_lista_anadir(init_test, end_test, nodo, tipo):
    '''
    Valida la operacion de añadir un elemento a la lista enlazada. 
    Se espera que el elemento se añada al final (contrato de los metodos de la estructura)
       
    Args:
        init_test: lista de elementos de la lista enlazada antes de ejecutar la operacion
        end_test: lista de elementos de la lista enlazada despues de ejecutar la operacion
        nodo: nodo que se añade a la lista enlazada
        tipo: tipo de lista enlazada (1: sencilla, 2: doble)
        
    Returns:
        state_val: Estado final de la validacion (WARNING, SUCCESSFUL, FAILED)
        end_val: lista de los elementos de la estructura de referencia al ejecutar la operacion en cuestion
        comment: mensaje informativo para el usuario
    '''
    structure_ref = referenciaLista(tipo)
    for i in init_test:                     #Inicializacion de la estructura de prueba
        structure_ref.addNode_byValue(i)
    structure_ref.addNode_byValue(nodo)     #Ejecutar accion
    end_val = structure_ref.getNodeValues() #Resultado de validacion
    
    if len(end_test) == len(end_val): # Hay la cantidad esperada de elementos
        sameOrder = True
        for i in range(len(end_test)):
            if end_test[i] != end_val[i]:
                sameOrder = False
                break                       
        if sameOrder:
            comment = 'El elemento "'+ str(nodo) + '" se añadió satisfactoriamente'
            state_val = VALIDATION_STATES[1]
        else:
            test = set(end_test)
            val = set(end_val)
            if test == val:
                comment = 'El elemento "'+ str(nodo) + '" se añadió satisfactoriamente, pero no en la posición esperada'
                state_val = VALIDATION_STATES[0]
            else:
                comment = 'El elemento "'+ str(nodo) + '" NO se añadió, hay un elemento adicional pero no corresponde al elemento esperado'
                state_val = VALIDATION_STATES[-1]
    elif len(end_test) == len(end_val) - 1: # no hay cambios en la cantidad de elementos
        sameOrder = True
        for i in range(len(end_test)):
            if end_test[i] != end_val[i]:
                sameOrder = False
                break
        if sameOrder:
            comment = 'No se añadió el elemento "' + str(nodo) + '"'
            state_val = VALIDATION_STATES[-1]
        else:
            comment = 'El elemento "'+ str(nodo) + '" pudo haberse añadido, pero reemplazó a un elemento de la lista'
            state_val = VALIDATION_STATES[-1]
    elif len(end_test) < len(init_test): # Se eliminaron elementos
        comment = 'Se eliminaron elementos de la lista, hay menos de los que habian antes de ejecutar la operación de añadir'
        state_val = VALIDATION_STATES[-1]
    else:   # Hay mas elementos de los esperados
        cant = len(end_test) - len(end_val)
        comment = 'Hay '+ str(cant) + ' elementos más de los esperados'
        state_val = VALIDATION_STATES[-1]
    return state_val, end_val, comment

def validar_lista_eliminar(init_test, end_test, nodo, tipo, ans):
    '''
    Valida la operacion de eliminar un elemento de la lista enlazada. 
    Se espera que el elemento se elimine de la lista.
       
    Args:
        init_test: lista de elementos de la lista enlazada antes de ejecutar la operacion
        end_test: lista de elementos de la lista enlazada despues de ejecutar la operacion
        nodo: nodo que se elimina de la lista enlazada
        tipo: tipo de lista enlazada (1: sencilla, 2: doble)
        ans: resultado (True/False) de ejecutar el metodo deleteNode_byValue() en la estructura de prueba
        
    Returns:
        state_val: Estado final de la validacion (WARNING, SUCCESSFUL, FAILED)
        end_val: lista de los elementos de la estructura de referencia al ejecutar la operacion en cuestion
        comment: mensaje informativo para el usuario
    '''
    structure_ref = referenciaLista(tipo)
    for i in init_test:                     #Inicializacion de la estructura de prueba
        structure_ref.addNode_byValue(i)
    ans2 = structure_ref.deleteNode_byValue(nodo)  #Ejecutar accion
    end_val = structure_ref.getNodeValues() #Resultado de validacion
      
    if len(end_test) == len(end_val): # Hay la cantidad esperada de elementos
        sameOrder = True
        for i in range(len(end_test)):
            if end_test[i] != end_val[i]:
                sameOrder = False
                break                       
        if sameOrder:
            comment = 'El elemento "'+ str(nodo) + '" se eliminó satisfactoriamente'
            state_val = VALIDATION_STATES[1]
        else:
            end_test = sorted(end_test, key=functools.cmp_to_key(defaultfunction))
            end_val = sorted(end_val, key=functools.cmp_to_key(defaultfunction))
            if(end_test == end_val):
                comment = 'Se eliminó una de las instancias del elemento "'+ str(nodo) + '"'
                state_val = VALIDATION_STATES[1]
            else:
                comment = 'El elemento "'+ str(nodo) + '" No se eliminó satisfactoriamente, se eliminó un elemento diferente'
                state_val = VALIDATION_STATES[-1]
    elif len(end_test) == len(init_test): # no hay cambios en la cantidad de elementos
        sameOrder = True
        for i in range(len(end_test)):
            if end_test[i] != init_test[i]:
                sameOrder = False
                break
        if sameOrder:
            comment = 'No se eliminó el elemento "' + str(nodo) + '". La lista no presenta cambios'
            state_val = VALIDATION_STATES[-1]
        else:
            comment = 'No se eliminó el elemento "' + str(nodo) + '". La lista presenta cambios no esperados'
            state_val = VALIDATION_STATES[-1]     
    elif len(end_test) < len(end_val): # Se eliminaron mas elementos
        test = set(end_test)
        val = set(end_val)
        if nodo not in end_test:
            comment = 'Se eliminaron todas las instancias del elemento "' + str(nodo) +'" cuando solo una de ellas se debia eliminar'
            state_val = VALIDATION_STATES[0]
        elif test == val:
            comment = 'Se eliminaron más instancias del elemento "' + str(nodo) +'" cuando solo una de ellas se debia eliminar'
            state_val = VALIDATION_STATES[-1]
        else:
            comment = 'Se eliminaron más elementos de los esperados'
            state_val = VALIDATION_STATES[-1]
    else:   # Hay mas elementos de los esperados
        cant = len(end_test) - len(end_val)
        comment = 'Hay '+ str(cant) + ' elementos más de los esperados'
        state_val = VALIDATION_STATES[-1]
    
    if ans != ans2:
        ans2 = 'El resultado de return del método es diferente al esperado. Se esperaba ' + str(ans2) + ' y se obtuvo ' + str(ans)
        comment = comment + '\n' + ans2   
    
    return state_val, end_val, comment

def validar_lista_encontrar(nodos, nodo, ans):
    '''
    Valida la operacion de encontrar un elemento en la lista enlazada.
       
    Args:
        nodos: lista de elementos de la lista enlazada antes de ejecutar la operacion
        nodo: nodo que se busca en la lista enlazada
        ans: resultado (True/False) de ejecutar el metodo isNodeValue() en la estructura de prueba
        
    Returns:
        state_val: Estado final de la validacion (WARNING, SUCCESSFUL, FAILED)
        end_val: lista de los elementos de la estructura de referencia al ejecutar la operacion en cuestion
        comment: mensaje informativo para el usuario
    '''
    if nodo in nodos:
        ans_val = True
    else:
        ans_val = False
    
    if ans == ans_val:
        if ans:
            comment = 'El elemento "'+ str(nodo) + '" si se encuentra en la lista'
        else:
            comment = 'El elemento "'+ str(nodo) + '" NO se encuentra en la lista'
        state_val = VALIDATION_STATES[1]
    else:
        comment = 'El elemento "'+ str(nodo) + '" si se encuentra en la lista pero el método isNodeVale() reporta que no se encuentra'
        state_val = VALIDATION_STATES[-1]
    
    return state_val, ans_val, comment

def validar_lista_adyacentes(init_test, listaAdj, nodo, tipo):
    '''
    Valida la operacion de encontrar los valores de los nodos adyacentes a un 
    elemento en la lista enlazada.
       
    Args:
        init_test: lista de elementos de la lista enlazada antes de ejecutar la operacion
        listaAdj: lista de valores de nodos adyacentes obtenida por la estructura de prueba
        nodo: nodo al cual se le buscan los adyacentes
        tipo: tipo de lista enlazada (1: sencilla, 2: doble)
               
    Returns:
        state_val: Estado final de la validacion (WARNING, SUCCESSFUL, FAILED)
        listaAdj_val: lista de los elementos adyacentes obtenidos por la estructura de referencia
        comment: mensaje informativo para el usuario
        exists: indica si el nodo existe en la lista
        
    '''
    structure_ref = referenciaLista(tipo)
    for i in init_test:                     #Inicializacion de la estructura de prueba
        structure_ref.addNode_byValue(i)
    
    listaAdj_val = structure_ref.findAdjacentNode(nodo)
    
    listaAdj_val = sorted(listaAdj_val, key=functools.cmp_to_key(defaultfunction))
    listaAdj = sorted(listaAdj, key=functools.cmp_to_key(defaultfunction))
    txt = ''
    for i in listaAdj_val:
        txt = txt + str(i) + ', '
        
    if listaAdj == listaAdj_val:
        comment = 'El elemento "'+ str(nodo)+ '" tiene '+str(len(listaAdj_val))+' adyacente(s): ' + txt[:-2]
        state_val = VALIDATION_STATES[1]
    else:
        comment = 'No se encontraron todos los adyacentes del elemento\nSe esperaban los elementos: ' + txt[:-2]
        txt = ''
        for i in listaAdj:
            txt = txt + str(i) + ', '
        comment = comment + '\nSe obtuvo: ' + txt[:-2]
        state_val = VALIDATION_STATES[-1]
    
    exists = structure_ref.isNodeValue(nodo)
        
    return state_val, listaAdj_val, comment, exists


### Validación - Árboles BST

In [9]:
def validar_bst_crear(nodos, st_nodos):
    '''
    Valida la operacion de crear un arbol BST
       
    Args:
        nodos: nodos que se añadieron al crear el arbol
        st_nodos: nodos que la estructura tiene
               
    Returns:
        state_val: Estado final de la validacion (WARNING, SUCCESSFUL, FAILED)
        comment: mensaje informativo para el usuario
        
    '''
    structure_ref = referenciaArbol()
    for i in nodos:                     #Inicializacion de la estructura de prueba
        structure_ref.addNode_byValue(i)
    
    nodos = structure_ref.getNodeValues("Preorder")
    
    txt_nodos = ''
    for i in nodos:
        txt_nodos = txt_nodos + str(i) + ', '
    
    txt_st = ''
    for i in st_nodos:
        txt_st = txt_st + str(i) + ', '
        
    if len(nodos) == len(st_nodos):
        if nodos == st_nodos:
            if len(nodos) != 0:
                comment = 'Nodos: ' + txt_nodos[:-2]
            else:
                comment = 'Nodos: []'
            state_val = VALIDATION_STATES[1]
        else:
            nodos = sorted(nodos, key=functools.cmp_to_key(defaultfunction))
            st_nodos = sorted(st_nodos, key=functools.cmp_to_key(defaultfunction))
            if nodos == st_nodos:
                comment = 'Se añadieron los elementos pero en un orden diferente'
                comment = comment + '\nSe esperaba: ' + txt_nodos[:-2]
                comment = comment + '\n  Se obtuvo: ' + txt_st[:-2]
                state_val = VALIDATION_STATES[0]
            else:
                comment = 'Se añadieron elementos diferentes a los indicados'
                comment = comment + '\nSe esperaba: ' + txt_nodos[:-2]
                comment = comment + '\n  Se obtuvo: ' + txt_st[:-2]
                state_val = VALIDATION_STATES[-1]
    else:
        comment = 'Se añadieron más elementos de los esperados'
        comment = comment + '\nSe esperaba: ' + txt_nodos[:-2]
        comment = comment + '\n  Se obtuvo: ' + txt_st[:-2]
        state_val = VALIDATION_STATES[-1]
    
    return state_val, comment

def validar_bst_anadir(init_test, end_test, nodo):
    '''
    Valida la operacion de añadir un elemento al arbol. 
    Se espera que el elemento se añada en la posicion correcta
       
    Args:
        init_test: lista de elementos del arbol antes de ejecutar la operacion
        end_test: lista de elementos del arbol despues de ejecutar la operacion
        nodo: nodo que se añade al arbol
        
    Returns:
        state_val: Estado final de la validacion (WARNING, SUCCESSFUL, FAILED)
        end_val: lista de los elementos en Preorder de la estructura de referencia al ejecutar la operacion en cuestion
        comment: mensaje informativo para el usuario
    '''
    structure_ref = referenciaArbol()
    for i in init_test:                     #Inicializacion de la estructura de prueba
        structure_ref.addNode_byValue(i)
    structure_ref.addNode_byValue(nodo)     #Ejecutar accion
    end_val = structure_ref.getNodeValues("Preorder") #Resultado de validacion
    
    if len(end_test) == len(end_val): # Hay la cantidad esperada de elementos
        sameOrder = True
        if len(init_test) == len(end_val):
            comment = 'El elemento "'+ str(nodo) + '" ya se encontraba en el bst'
            state_val = VALIDATION_STATES[1]
        else:
            for i in range(len(end_test)):
                if end_test[i] != end_val[i]:
                    sameOrder = False
                    break                       
            if sameOrder:
                comment = 'El elemento "'+ str(nodo) + '" se añadió satisfactoriamente'
                state_val = VALIDATION_STATES[1]
            else:
                test = set(end_test)
                val = set(end_val)
                if test == val:
                    comment = 'El elemento "'+ str(nodo) + '" se añadió satisfactoriamente, pero no en la posición esperada'
                    state_val = VALIDATION_STATES[0]
                else:
                    comment = 'El elemento "'+ str(nodo) + '" NO se añadió, hay un elemento adicional pero no corresponde al elemento esperado'
                    state_val = VALIDATION_STATES[-1]
    elif len(end_test) == len(end_val) - 1: # no hay cambios en la cantidad de elementos
        sameOrder = True
        for i in range(len(end_test)):
            if end_test[i] != end_val[i]:
                sameOrder = False
                break
        if sameOrder:
            comment = 'No se añadió el elemento "' + str(nodo) + '"'
            state_val = VALIDATION_STATES[-1]
        else:
            comment = 'El elemento "'+ str(nodo) + '" pudo haberse añadido, pero reemplazó a un elemento de la lista'
            state_val = VALIDATION_STATES[-1]
    elif len(end_test) < len(init_test): # Se eliminaron elementos
        comment = 'Se eliminaron elementos de la lista, hay menos de los que habian antes de ejecutar la operación de añadir'
        state_val = VALIDATION_STATES[-1]
    else:   # Hay mas elementos de los esperados
        cant = len(end_test) - len(end_val)
        comment = 'Hay '+ str(cant) + ' elementos más de los esperados'
        state_val = VALIDATION_STATES[-1]
    
    return state_val, end_val, comment

def validar_bst_eliminar(init_test, end_test, nodo, ans):
    '''
    Valida la operacion de eliminar un elemento del arbol. 
       
    Args:
        init_test: lista de elementos del arbol antes de ejecutar la operacion
        end_test: lista de elementos del arbol despues de ejecutar la operacion
        nodo: nodo que se elimina del arbol
        ans: resultado (True/False) de ejecutar el metodo deleteNode_byValue() en la estructura de prueba
        
    Returns:
        state_val: Estado final de la validacion (WARNING, SUCCESSFUL, FAILED)
        end_val: lista de los elementos de la estructura de referencia al ejecutar la operacion en cuestion
        comment: mensaje informativo para el usuario
    '''
    structure_ref = referenciaArbol()
    for i in init_test:                     #Inicializacion de la estructura de prueba
        structure_ref.addNode_byValue(i)
    ans2 = structure_ref.deleteNode_byValue(nodo)  #Ejecutar accion
    end_val = structure_ref.getNodeValues("Preorder") #Resultado de validacion
      
    if len(end_test) == len(end_val): # Hay la cantidad esperada de elementos
        sameOrder = True
        for i in range(len(end_test)):
            if end_test[i] != end_val[i]:
                sameOrder = False
                break                       
        if sameOrder and end_test != init_test:
            comment = 'El elemento "'+ str(nodo) + '" se eliminó satisfactoriamente'
            state_val = VALIDATION_STATES[1]
        else:
            comment = 'El elemento "'+ str(nodo) + '" No se pertenece al BST'
            state_val = VALIDATION_STATES[0]
    elif len(end_test) == len(init_test): # no hay cambios en la cantidad de elementos
        sameOrder = True
        for i in range(len(end_test)):
            if end_test[i] != init_test[i]:
                sameOrder = False
                break
        if sameOrder:
            comment = 'No se eliminó el elemento "' + str(nodo) + '". La estructura no presenta cambios'
            state_val = VALIDATION_STATES[-1]
        else:
            comment = 'No se eliminó el elemento "' + str(nodo) + '". La estructura presenta cambios no esperados'
            state_val = VALIDATION_STATES[-1]     
    elif len(end_test) < len(end_val): # Se eliminaron mas elementos
        test = set(end_test)
        val = set(end_val)
        if nodo not in end_test:
            comment = 'Se eliminaron todas las instancias del elemento "' + str(nodo) +'" cuando solo una de ellas se debia eliminar'
            state_val = VALIDATION_STATES[0]
        elif test == val:
            comment = 'Se eliminaron más instancias del elemento "' + str(nodo) +'" cuando solo una de ellas se debia eliminar'
            state_val = VALIDATION_STATES[-1]
        else:
            comment = 'Se eliminaron más elementos de los esperados'
            state_val = VALIDATION_STATES[-1]
    else:   # Hay mas elementos de los esperados
        cant = len(end_test) - len(end_val)
        comment = 'Hay '+ str(cant) + ' elementos más de los esperados'
        state_val = VALIDATION_STATES[-1]
    
    if ans != ans2:
        ans2 = 'El resultado de return del método es diferente al esperado. Se esperaba ' + str(ans2) + ' y se obtuvo ' + str(ans)
        comment = comment + '\n' + ans2   
    return state_val, end_val, comment

def validar_bst_encontrar(nodos, nodo, ans):
    '''
    Valida la operacion de encontrar un elemento en el arbol.
       
    Args:
        nodos: lista de elementos del arbol antes de ejecutar la operacion
        nodo: nodo que se busca en el arbol
        ans: resultado (True/False) de ejecutar el metodo isNodeValue() en la estructura de prueba
        
    Returns:
        state_val: Estado final de la validacion (WARNING, SUCCESSFUL, FAILED)
        end_val: lista de los elementos de la estructura de referencia al ejecutar la operacion en cuestion
        comment: mensaje informativo para el usuario
    '''
    if nodo in nodos:
        ans_val = True
    else:
        ans_val = False
    
    if ans == ans_val:
        if ans:
            comment = 'El elemento "'+ str(nodo) + '" si se encuentra en el arbol'
        else:
            comment = 'El elemento "'+ str(nodo) + '" NO se encuentra en el arbol'
        state_val = VALIDATION_STATES[1]
    else:
        comment = 'El elemento "'+ str(nodo) + '" si se encuentra en la estructura pero el método isNodeVale() reporta que no se encuentra'
        state_val = VALIDATION_STATES[-1]
    
    return state_val, ans_val, comment

def validar_bst_adyacentes(init_test, listaAdj, nodo):
    '''
    Valida la operacion de encontrar los valores de los nodos adyacentes a un 
    elemento en el grafo.
       
    Args:
        init_test: lista de elementos del grafo antes de ejecutar la operacion
        listaAdj: lista de valores de nodos adyacentes obtenida por la estructura de prueba
        nodo: nodo al cual se le buscan los adyacentes
               
    Returns:
        state_val: Estado final de la validacion (WARNING, SUCCESSFUL, FAILED)
        listaAdj_val: lista de los elementos adyacentes obtenidos por la estructura de referencia
        comment: mensaje informativo para el usuario
        exists: indica si el nodo existe en la lista
        
    '''
    structure_ref = referenciaArbol()
    for i in init_test:                     #Inicializacion de la estructura de prueba
        structure_ref.addNode_byValue(i)
    listaAdj_val = structure_ref.findAdjacentNode(nodo)   
    
    listaAdj_val = sorted(listaAdj_val, key=functools.cmp_to_key(defaultfunction))
    listaAdj = sorted(listaAdj, key=functools.cmp_to_key(defaultfunction))
    txt = ''
    for i in listaAdj_val:
        txt = txt + str(i) + ', '

    if listaAdj == listaAdj_val:
        comment = 'El elemento "'+ str(nodo)+ '" tiene '+str(len(listaAdj_val))+' adyacente(s): ' + txt[:-2]
        state_val = VALIDATION_STATES[1]
    else:
        comment = 'No se encontraron todos los adyacentes del elemento\nSe esperaban los elementos: ' + txt[:-2]
        txt = ''
        for i in listaAdj:
            txt = txt + str(i) + ', '
        comment = comment + '\nSe obtuvo: ' + txt[:-2]
        state_val = VALIDATION_STATES[-1]
    
    exists = structure_ref.isNodeValue(nodo)
    
    return state_val, listaAdj_val, comment, exists

def validar_bst_darNodos(init_test, nodos, orden):
    '''
    Valida la operacion de encontrar listar los valores de los nodos de un arbol dado un orden
       
    Args:
        init_test: lista de elementos del arbol en preorder
        nodos: lista de valores de nodos obtenida de listar los nodos en el orden dado
        orden: orden en el cual se listan los nodos (preorder, inorder, postorder)
               
    Returns:
        state_val: Estado final de la validacion (WARNING, SUCCESSFUL, FAILED)
        nodos_val: lista de los elementos en el orden dado, en la estructura de referencia
        comment: mensaje informativo para el usuario
        
    '''
    structure_ref = referenciaArbol()
    for i in init_test:                     #Inicializacion de la estructura de prueba
        structure_ref.addNode_byValue(i)
    
    nodos_val = structure_ref.getNodeValues(orden)
    
    txt_val = ''
    for i in nodos_val:
        txt_val = txt_val + str(i) + ', '
        
    txt_test = ''
    for i in nodos:
        txt_test = txt_test + str(i) + ', '
    
    if nodos_val == nodos:
        comment = 'Se obtuvo de manera exitosa los elementos en ' + orden + '\nElementos: ' + txt_val[:-2]
        state_val = VALIDATION_STATES[1]
    else:
        nodos_val = sorted(nodos_val, key=functools.cmp_to_key(defaultfunction))
        nodos = sorted(nodos, key=functools.cmp_to_key(defaultfunction))
        if nodos == nodos_val:
            comment = 'No se obtuvieron los elementos en el orden correcto (' + orden + '):'
            comment = comment + '\nSe esperaba: ' + txt_val[:-2]
            comment = comment + '\n  Se obtuvo: ' + txt_test[:-2]
            state_val = VALIDATION_STATES[-1]
        else:
            comment = 'Hay una cantidad diferente de elementos de la esperada (' + orden + '):'
            comment = comment + '\nSe esperaba: ' + txt_val[:-2]
            comment = comment + '\n  Se obtuvo: ' + txt_test[:-2]
            state_val = VALIDATION_STATES[-1]            
    return state_val, nodos_val, comment


### Validación - Grafos

In [10]:
def validar_graph_crear(nodos, edges, nodos_test, edges_test, tipo):
    '''
    Valida la operacion de crear un grafo dado un tipo
       
    Args:
        nodos: nodos que se añadieron al crear el grafo
        edges: arcos que se añadieron al crear el grafo
        nodos_test: nodos que la estructura tiene
        edges_test: arcos que tiene el grafo
        tipo: tipo del grafo (4: Dirigido, 5: No dirigido)
               
    Returns:
        state_val: Estado final de la validacion (WARNING, SUCCESSFUL, FAILED)
        comment: mensaje informativo para el usuario
        
    '''
    if tipo == 4:
        structure_ref = referenciaGrafo('Directed')
    else:
        structure_ref = referenciaGrafo()
    for i in nodos:                     #Inicializacion de la estructura de prueba
        structure_ref.addNode_byValue(i)
    nodos = structure_ref.getNodeValues()
    
    for i,j,k in edges:                     #Inicializacion de la estructura de prueba
        structure_ref.addEdge_byValue(i,j,k)
    edges = structure_ref.getEdgeValues()
    
    txt_nodos_ref = ''
    for i in nodos:
        txt_nodos_ref = txt_nodos_ref + str(i) + ', '
    txt_nodos_test = ''
    for i in nodos_test:
        txt_nodos_test = txt_nodos_test + str(i) + ', '
        
    validation = 0    
    state_val = VALIDATION_STATES[0]
    comment = ''
    
    if len(nodos) == len(nodos_test):
        nodos = sorted(nodos, key=functools.cmp_to_key(defaultfunction))
        nodos_test = sorted(nodos_test, key=functools.cmp_to_key(defaultfunction))
        if nodos == nodos_test:
            validation = validation + 1
        else:
            comment = 'No se tienen los vertices que se esperan tener'
            comment = comment + '\nSe esperaba: ' + txt_nodos_ref[:-2]
            comment = comment + '\n  Se obtuvo: ' + txt_nodos_test[:-2]
            state_val = VALIDATION_STATES[-1]
    else:
        comment = 'Se tiene una cantidad diferente de vertices a los esperados'
        comment = comment + '\nSe esperaba: ' + txt_nodos_ref[:-2]
        comment = comment + '\n  Se obtuvo: ' + txt_nodos_test[:-2]
        state_val = VALIDATION_STATES[-1]
        
    txt_edges_ref = ''
    for i in edges:
        txt_edges_ref = txt_edges_ref + str(i) + ', '
    txt_edges_test = ''
    for i in edges_test:
        txt_edges_test = txt_edges_test + str(i) + ', '
        
    if len(edges) == len(edges_test):
        edges.sort()
        edges_test.sort()
        if edges == edges_test:
            validation = validation + 1
        else:
            comment = comment + '\nNo se tienen los arcos que se esperan tener'
            comment = comment + '\nSe esperaba: ' + txt_nodos_ref[:-2]
            comment = comment + '\n  Se obtuvo: ' + txt_nodos_test[:-2]
            state_val = VALIDATION_STATES[-1]
    else:
        comment = comment + '\nSe tiene una cantidad diferente de arcos a los esperados'
        comment = comment + '\nSe esperaba: ' + txt_nodos_ref[:-2]
        comment = comment + '\n  Se obtuvo: ' + txt_nodos_test[:-2]
        state_val = VALIDATION_STATES[-1]
        
    if validation == 2:
        comment = 'El grafo se creó satisfactoriamente'
        state_val = VALIDATION_STATES[1]
    
    return state_val, comment   

def validar_graph_anadir(init_test, end_test, tipo, nodo):
    '''
    Valida la operacion de un nodo al grafo
       
    Args:
        init_test: nodos que la estructura tiene antes de añadir el nuevo nodo
        end_test: nodos que la estructura tiene despues de añadir el nuevo nodo
        tipo: tipo del grafo (4: Dirigido, 5: No dirigido)
        nodo: nodo que se añade al grafo               
    Returns:
        state_val: Estado final de la validacion (WARNING, SUCCESSFUL, FAILED)
        comment: mensaje informativo para el usuario
        
    '''
    if tipo == 4:
        structure_ref = referenciaGrafo('Directed')
    else:
        structure_ref = referenciaGrafo()
    for i in init_test:                     #Inicializacion de la estructura de prueba
        structure_ref.addNode_byValue(i)
    structure_ref.addNode_byValue(nodo)
    end_ref = structure_ref.getNodeValues()
    
    txt_nodos_ref = ''
    for i in end_ref:
        txt_nodos_ref = txt_nodos_ref + str(i) + ', '
    txt_nodos_test = ''
    for i in end_test:
        txt_nodos_test = txt_nodos_test + str(i) + ', '
        
    state_val = VALIDATION_STATES[0]
    comment = ''
    
    if len(end_ref) == len(end_test):
        end_ref = sorted(end_ref, key=functools.cmp_to_key(defaultfunction))
        end_test = sorted(end_test, key=functools.cmp_to_key(defaultfunction))
        init_test = sorted(init_test, key=functools.cmp_to_key(defaultfunction))
        if end_test == end_ref and end_ref != init_test:
            comment = 'El vertice se añadió correctamente'
            state_val = VALIDATION_STATES[1]
        elif end_test == end_ref:
            comment = 'El vertice ya se encuentra en el grafo'
            state_val = VALIDATION_STATES[1]
        else:
            comment = 'No se tienen los vertices que se esperan tener'
            comment = comment + '\nSe esperaba: ' + txt_nodos_ref[:-2]
            comment = comment + '\n  Se obtuvo: ' + txt_nodos_test[:-2]
            state_val = VALIDATION_STATES[-1]
    else:
        comment = 'Se tiene una cantidad diferente de vertices a los esperados'
        comment = comment + '\nSe esperaba: ' + txt_nodos_ref[:-2]
        comment = comment + '\n  Se obtuvo: ' + txt_nodos_test[:-2]
        state_val = VALIDATION_STATES[-1]
    return state_val, comment   

def validar_graph_eliminar(init_test, end_test, tipo, nodo):
    '''
    Valida la operacion de un eliminar un nodo del grafo
       
    Args:
        init_test: nodos que la estructura tiene antes de eliminar el nodo
        end_test: nodos que la estructura tiene despues de eliminar el nodo
        tipo: tipo del grafo (4: Dirigido, 5: No dirigido)
        nodo: nodo que se elimina del grafo               
    Returns:
        state_val: Estado final de la validacion (WARNING, SUCCESSFUL, FAILED)
        comment: mensaje informativo para el usuario
        
    '''
    if tipo == 4:
        structure_ref = referenciaGrafo('Directed')
    else:
        structure_ref = referenciaGrafo()
    for i in init_test:                     #Inicializacion de la estructura de prueba
        structure_ref.addNode_byValue(i)
    structure_ref.deleteNode_byValue(nodo)
    end_ref = structure_ref.getNodeValues()
    
    txt_nodos_ref = ''
    for i in end_ref:
        txt_nodos_ref = txt_nodos_ref + str(i) + ', '
    txt_nodos_test = ''
    for i in end_test:
        txt_nodos_test = txt_nodos_test + str(i) + ', '
        
    state_val = VALIDATION_STATES[0]
    comment = ''
    
    if len(end_ref) == len(end_test):
        end_ref = sorted(end_ref, key=functools.cmp_to_key(defaultfunction))
        end_test = sorted(end_test, key=functools.cmp_to_key(defaultfunction))
        init_test = sorted(init_test, key=functools.cmp_to_key(defaultfunction))
        if end_test == end_ref and end_ref != init_test:
            comment = 'El vertice se eliminó correctamente'
            state_val = VALIDATION_STATES[1]
        elif end_test == end_ref:
            comment = 'El vertice NO se encuentra en el grafo'
            state_val = VALIDATION_STATES[0]
        else:
            comment = 'No se tienen los vertices que se esperan tener'
            comment = comment + '\nSe esperaba: ' + txt_nodos_ref[:-2]
            comment = comment + '\n  Se obtuvo: ' + txt_nodos_test[:-2]
            state_val = VALIDATION_STATES[-1]
    else:
        comment = 'Se tiene una cantidad diferente de vertices a los esperados'
        comment = comment + '\nSe esperaba: ' + txt_nodos_ref[:-2]
        comment = comment + '\n  Se obtuvo: ' + txt_nodos_test[:-2]
        state_val = VALIDATION_STATES[-1]
    
    return state_val, comment   

def validar_graph_existe(init_test, existe_test, nodo):
    '''
    Valida la operacion de un verifcar si un nodo pertenece al grafo
       
    Args:
        init_test: nodos que la estructura de prueba tiene
        existe_test: resultado de la verificacion por parte de la estructura de prueba
        nodo: nodo que se verifica la pertenencia al grafo               
    Returns:
        state_val: Estado final de la validacion (WARNING, SUCCESSFUL, FAILED)
        comment: mensaje informativo para el usuario
        
    '''
    existe_ref = str(nodo) in init_test
    
    state_val = VALIDATION_STATES[0]
    comment = ''
    
    if existe_test == existe_ref:
        if existe_test:
            comment = 'El vertice "' + str(nodo) + '" SI se encuentra en el grafo'
        else:
            comment = 'El vertice "' + str(nodo) + '" NO se encuentra en el grafo'
        state_val = VALIDATION_STATES[1]
    else:
        if existe_ref:
            comment = 'El vertice Si se encuentra en el grafo, pero la estructura indica que no'
        else:
            comment = 'El vertice NO se encuentra en el grafo, pero la estructura indica que si'
        state_val = VALIDATION_STATES[-1]
    
    return state_val, comment   

def validar_graph_anadirEdge(nodes, init_test, end_test, tipo, origen, destino, peso):
    '''
    Valida la operacion de un añadir un arco al grafo
       
    Args:
        nodes: nodos de la estructura de prueba
        init_test: arcos que la estructura de prueba tiene antes de añadir el arco
        end_test: arcos que la estructura de prueba tiene despues de añadir el arco
        tipo: tipo del grafo (4:dirigido, 5:no dirigido)
        origen: nodo origen del arco
        destino: nodo destino del arco
        peso: peso del arco
        
    Returns:
        state_val: Estado final de la validacion (WARNING, SUCCESSFUL, FAILED)
        comment: mensaje informativo para el usuario
        
    '''
    if tipo == 4:
        structure_ref = referenciaGrafo('Directed')
    else:
        structure_ref = referenciaGrafo()
    for i in nodes:                     #Inicializacion de la estructura de prueba
        structure_ref.addNode_byValue(i)
    for i,j,k in init_test:
        structure_ref.addEdge_byValue(i,j,k)
        
    structure_ref.addEdge_byValue(origen,destino,peso)
    end_ref = structure_ref.getEdgeValues()

    txt_edges_ref = ''
    for i in end_ref:
        txt_edges_ref = txt_edges_ref + str(i) + ', '
    txt_edges_test = ''
    for i in end_test:
        txt_edges_test = txt_edges_test + str(i) + ', '
        
    if len(end_ref) == len(end_test):
        end_ref.sort()
        end_test.sort()
        if end_test == end_ref:
            arco = '(' + str(origen) + ' -> ' + str(destino) + ',' + str(peso) + ')'
            comment = 'Se añadió el arco "' + arco + '" correctamente'
            state_val = VALIDATION_STATES[1]
        else:
            comment = 'No se tienen los arcos que se esperan tener'
            comment = comment + '\nSe esperaba: ' + txt_edges_ref[:-2]
            comment = comment + '\n  Se obtuvo: ' + txt_edges_test[:-2]
            state_val = VALIDATION_STATES[-1]
    else:
        comment = 'Se tiene una cantidad diferente de Arcos a los esperados'
        comment = comment + '\nSe esperaba: ' + txt_edges_ref[:-2]
        comment = comment + '\n  Se obtuvo: ' + txt_edges_test[:-2]
        state_val = VALIDATION_STATES[-1]
    
    return state_val, comment  

def validar_graph_adj(nodes, edges, adjNodes, nodo, tipo):
    '''
    Valida la operacion de un encontrar los adyacentes de u nodo en el grafo
       
    Args:
        nodes: nodos de la estructura de prueba
        edges: arcos de la estructura de prueba
        adjNodes: nodos que la estructura de prueba determina como adyacentes de nodo
        nodo: nodo del que se buscan los adyacentes
        tipo: tipo del grafo (4:dirigido, 5:no dirigido)
        
    Returns:
        state_val: Estado final de la validacion (WARNING, SUCCESSFUL, FAILED)
        comment: mensaje informativo para el usuario
        
    '''
    #---------Inicializacion de la estructura de prueba
    if tipo == 4:
        structure_ref = referenciaGrafo('Directed')
    else:
        structure_ref = referenciaGrafo()
    for i in nodes:                     
        structure_ref.addNode_byValue(i)
    for i,j,k in edges:
        structure_ref.addEdge_byValue(i,j,k)    
    adjNodes_ref = structure_ref.findAdjacentNode(nodo)
    aux = structure_ref.findAdjacentNode(nodo)
    #---------
    if nodo not in nodes:
        comment = 'El nodo "' +str(nodo) + '" no se encuentra en el grafo'
        state_val = VALIDATION_STATES[0]
    else:
        adjNodes = list(dict.fromkeys(adjNodes))
        adjNodes_ref = list(dict.fromkeys(adjNodes_ref))
        
        adjText = ''
        for i in adjNodes:
            adjText = adjText + str(i) + ', '
        adjText_ref = ''
        for i in adjNodes_ref:
            adjText_ref = adjText_ref + str(i) + ', '
        
        if len(adjNodes) == len(adjNodes_ref):
            adjNodes.sort()
            adjNodes_ref.sort()
            if adjNodes == adjNodes_ref:
                comment = 'El nodo "' + str(nodo) + '" tiene ' + str(len(adjNodes)) + ' adyacentes: ' + adjText[:-2]
                state_val = VALIDATION_STATES[1]
            else:
                comment = 'Los adyacentes del nodos no son los esperados'
                comment = comment + '\nSe esperaba: ' + adjText_ref[:-2]
                comment = comment + '\n  Se obtuvo: ' + adjText[:-2]
                state_val = VALIDATION_STATES[-1]
        else:
            for i in adjNodes_ref:
                if i not in nodes:
                    aux.remove(i)
            adjNodes_ref = aux
            
            adjNodes.sort()
            adjNodes_ref.sort()
            if adjNodes == adjNodes_ref:
                comment = 'El nodo "' + str(nodo) + '" tiene ' + str(len(adjNodes)) + ' adyacentes: ' + adjText[:-2]
                state_val = VALIDATION_STATES[1]
            else:
                comment = 'Los adyacentes del nodos no son los esperados'
                comment = comment + '\nSe esperaba: ' + adjText_ref[:-2]
                comment = comment + '\n  Se obtuvo: ' + adjText[:-2]
                state_val = VALIDATION_STATES[-1]

    return state_val, comment 

def validar_graph_todos(nodes, tipo):
    '''
    Valida la operacion de un encontrar todos los nodos de un grafo
       
    Args:
        nodes: nodos de la estructura de prueba
        tipo: tipo del grafo (4:dirigido, 5:no dirigido)
        
    Returns:
        state_val: Estado final de la validacion (WARNING, SUCCESSFUL, FAILED)
        comment: mensaje informativo para el usuario
        
    '''
    #---------Inicializacion de la estructura de prueba
    if tipo == 4:
        structure_ref = referenciaGrafo('Directed')
    else:
        structure_ref = referenciaGrafo()
    for i in nodes:                     
        structure_ref.addNode_byValue(i)
    
    nodes_ref = structure_ref.getNodeValues()
    
    txt_nodos_ref = ''
    for i in nodes_ref:
        txt_nodos_ref = txt_nodos_ref + str(i) + ', '
    txt_nodos_test = ''
    for i in nodes:
        txt_nodos_test = txt_nodos_test + str(i) + ', '
        
    nodes_ref.sort()
    nodes.sort()
    if nodes_ref == nodes:
        comment = 'Hay ' + str(len(nodes)) + ' nodos en el grafo: ' + txt_nodos_ref[:-2]
        state_val = VALIDATION_STATES[1]
    else:
        comment = 'No se encontraron todos los nodos del grafo'
        comment = comment + '\n Se esperaban: ' + txt_nodos_ref[:-2]
        comment = comment + '\n    Se obtuvo: ' + txt_nodos_test[:-2]
        state_val = VALIDATION_STATES[-1]
    return state_val, comment

def validarRecorridosGrafo(estructura, tipo, tipo_inData, recorrido, result_test, nodo=None): #TODO
    '''
    Valida los resultados de ejecutar algoritmos sobre el grafo de prueba
       
    Args:
        estructura: grafo de prueba
        tipo: tipo del grafo (4:dirigido, 5:no dirigido)
        tipo_inData: tipo de la salida del argoritmo
        recorrido: algoritmo que se aplica
        result_test: resultado de ejecutar el algoritmo
        nodo: nodo de inicio del algoritmo
        
    Returns:
        state_val: Estado final de la validacion (WARNING, SUCCESSFUL, FAILED)
        comment: mensaje informativo para el usuario
        
    '''
    state_val = ''
    comment = ''
    nodos = estructura.getNodeValues()
    edges = estructura.getEdgeValues()
    # Inicializacion de la estructura
    if tipo == 4:
        structure_ref = referenciaGrafo('Directed')
    else:
        structure_ref = referenciaGrafo()
    for i in nodos:                     
        structure_ref.addNode_byValue(i)
    for i,j,k in edges:
        structure_ref.addEdge_byValue(i,j,k)  
    # ---
    
    # Aplicar el recorrido
    result_ref = structure_ref.algorithms(recorrido, nodo)
    refText = ''
    testText = ''
    if tipo_inData == 'lista_nodos':
        for i in result_ref:
            refText = refText + str(i) + ', '
        for i in result_test:
            testText = testText + str(i) + ', '       
        refText = refText[:-2]     
        testText = testText[:-2]    
    
    if tipo_inData == 'tupla':
        testEdges = result_test[0]
        testWeight = result_test[1]
        
        refEdges = result_ref[0]
        refWeight = result_ref[1]
        
        for i in refEdges:
            refText = refText + '(' + i[0] + '->' + i[1] + '), '
        for i in testEdges:
            testText = testText + '(' + i[0] + '->' + i[1] + '), '
        
        refText = refText[:-2] + '\n  COSTO: ' + str(refWeight) + '\n'
        testText = testText[:-2] + '\n  COSTO: ' + str(testWeight)
        
    if tipo_inData == 'edges':
        for i in result_ref:
            refText = refText + '(' + i[0] + '->' + i[1] + '), '
        for i in result_test:
            testText = testText + '(' + i[0] + '->' + i[1] + '), '
        refText = refText[:-2]     
        testText = testText[:-2]    
    
    if tipo_inData == 'dicts':
        for aux_dict in result_ref:
            refText = refText + '\n* "'+nodo+'"' +'-->' +'"'+str(aux_dict['node'])+'"' + ' Costo: ' + aux_dict['node'] + '\n  Path:'
            for i,j in aux_dict['path']:
                refText = refText + '(' + i + '->' + j + '), '
            if len(aux_dict['path'])>0:
                refText = refText[:-2]   
        for aux_dict in result_test:
            testText = testText + '\n* "'+nodo+'"' +'-->' +'"'+str(aux_dict['node'])+'"' + ' Costo: ' + aux_dict['node'] + '\n  Path:'
            for i,j in aux_dict['path']:
                testText = testText + '(' + i + '->' + j + '), '
            if len(aux_dict['path'])>0:
                testText = testText[:-2]  
    
    if tipo_inData == 'single_dict':
        for lista in result_ref.values():
            refText = refText + '\n* '
            for i in lista:
                refText = refText + i + ', '
            refText = refText[:-2]

        for lista in result_test.values():
            testText = testText + '\n* '
            for i in lista:
                testText = testText + i + ', '
            testText = testText[:-2]
                
    if nodo != None:
        comment = 'El algoritmo ' + recorrido + ' se ha ejecutado desde el nodo "'+ nodo + '"\n'
    else:
        comment = 'El algoritmo ' + recorrido + ' se ha ejecutado:'
    comment = comment + '\nSe esperaba: ' + refText
    comment = comment + '\n\nSe obtuvo: ' + testText    
    return state_val, comment


## 4. Componente de Enlace

En estas secciones es donde se hacen los llamados a los metodos de las estructuras de pruebas (archivos extrenos). Se ejecutan las operaciones, se validan, se hace manejo de errores y se muestran los resultados en el Canvas de la aplicación

### Enlace - Listas Enlazadas

In [11]:
def crearListaEnlazada(tipo, file, init, data={}):
    """
    Crea una lista enlazada

    Args:
        tipo: Sencilla o Doble
        file: Estructura de datos externa
        init: Vacia o Random
        data: JSON con informacion de creacion (nodos)
    Returns:
        La lista enlazada
    Raises:
        Exception
    """
    if tipo == 1: txt = 'Sencilla - ' + init
    else: txt = 'Doble - ' + init
    
    txtNodos = ''
    try:
        estructura = file.listaEnlazada(tipo)
    except:
        e = "\tProblema al crear la lista enlazada, método listaEnlazada()"
        raise Exception(e)
    long = 0
    nodos = list()
    if init == 'Random':
        txtNodos = '\tValores: '
        long = random.randint(5,15)
        nodos = create_n_random(long)
        for i in nodos:
            txtNodos = txtNodos + str(i) + ', '
            try:
                estructura.addNode_byValue(i)
            except:
                e = '\tProblema al añadir el elemento "'+str(i)+'", método addNode_byValue()'
                raise Exception(e)
    elif init == 'Estática' or init == 'Archivo':
        try:
            txtNodos = '\tValores: '
            nodos = data["valores"]
        except:
            raise Exception("El formato del archivo ingresado no es válido")
        for i in nodos:
            txtNodos = txtNodos + str(i) + ', '
            try:
                estructura.addNode_byValue(i)
            except:
                e = '\tProblema al añadir el elemento "'+str(i)+'", método addNode_byValue()'
                raise Exception(e)
    displayList(estructura, tipo) 
    
    try:
        st_nodos = estructura.getNodeValues()
    except:
        e = '\tProblema al obtener todos los nodos, método getNodeValues()'
        raise Exception(e)
    state_val, comment = validar_lista_crear(nodos, st_nodos, tipo)
    
    print('Crear Lista Enlazada', txt)
    print(state_val)
    print(comment)
    return estructura

def anadirNodoLista(estructura, tipo, nodo):
    """
    Añade un nodo a lista

    Args:
        estructura: lista enlazada
        tipo: tipo de lista (1: sencilla, 2:doble)
        nodo: valor del nodo
    Returns:
        -
    Raises:
        Exception
    """
    try:
        init_test = estructura.getNodeValues()
    except:
        e = '\tProblema al obtener todos los nodos, método getNodeValues()'
        raise Exception(e)
    try:
        estructura.addNode_byValue(nodo)
        end_test = estructura.getNodeValues()
        state_val, end_val, comment = validar_lista_anadir(init_test, end_test, nodo, tipo) 
    except:
        e = '\tProblema al añadir el elemento "'+str(nodo)+'", método addNode_byValue()'
        raise Exception(e)
    
    out.clear_output()
    displayList(estructura, tipo)
    print('Añadir elemento')
    
    print(state_val, comment)
    if state_val != VALIDATION_STATES[1]: # Si no fue exitoso, mostrar los resultados esperados y obtenidos
        print('Se esperaba:', end_val)
        print('Se obtuvo:  ', end_test)

def eliminarNodoLista(estructura, tipo, nodo):
    """
    Elimina un nodo de la lista

    Args:
        estructura: lista enlazada
        tipo: tipo de la lista (1:sencilla, 2:doble)
        nodo: valor del nodo
    Returns:
        -
    Raises:
        Exception
    """
    try:
        init_test = estructura.getNodeValues()
    except:
        e = '\tProblema al obtener todos los nodos, método getNodeValues()'
        raise Exception(e)
    try:
        ans = estructura.deleteNode_byValue(nodo)
        end_test = estructura.getNodeValues()
        state_val, end_val, comment = validar_lista_eliminar(init_test, end_test, nodo, tipo, ans) 
    except:
        e = '\tProblema al eliminar el elemento "'+str(nodo)+'", método deleteNode_byValue()'
    out.clear_output()
    displayList(estructura, tipo)
    print('Eliminar elemento')
    
    print(state_val, comment)
    if state_val != VALIDATION_STATES[1]: # Si no fue exitoso, mostrar los resultados esperados y obtenidos
        print('Se esperaba:', end_val)
        print('Se obtuvo:  ', end_test)

def encontrarNodoLista(estructura, tipo, nodo):
    """
    Encuentra un nodo en la estrcutura

    Args:
        estructura: lista enlazada
        tipo: tipo de la lista (1:sencilla, 2:doble)
        nodo: valor del nodo
    Returns:
        -
    Raises:
        Exception
    """
    try:
        ans = estructura.isNodeValue(nodo)
        nodos = estructura.getNodeValues()
        state_val, ans_val, comment = validar_lista_encontrar(nodos, nodo, ans) 
    except:
        e = '\tProblema al buscar el elemento "' + str(nodo) + '", método isNodeValue()'
        raise Exception(e)
    out.clear_output()
    lista = list()
    lista.append(nodo)
    if ans_val:
        displayList(estructura, tipo, lista)
    else:    
        displayList(estructura, tipo)
    
    print(state_val, comment)

def findAdjacentNodeLista(estructura, tipo, nodo):
    """
    Encuentra los adyacentes de un nodo a lista

    Args:
        estructura: lista enlazada
        tipo: tipo de la lista (1:sencilla, 2:doble)
        nodo: valor del nodo
    Returns:
        -
    Raises:
        Exception
    """
    try:
        init_test = estructura.getNodeValues()
    except:
        e = '\tProblema al obtener todos los nodos, método getNodeValues()'
        raise Exception(e)
    try:
        listaAdj = estructura.findAdjacentNode(nodo)
        state_val, listaAdj_val, comment, exists = validar_lista_adyacentes(init_test, listaAdj, nodo, tipo) 
    except:
        e = '\tProblema al buscar los adyacentes del elemento "' + str(nodo) + '" , método findAdjacentNode()'
        raise Exception(e)
    
    out.clear_output()
    displayList(estructura, tipo, listaAdj)
    
    if not exists:
        print('Encontrar Adyacentes\n\tEl elemento "'+str(nodo)+ '" no existe en la lista')
    else:
        print('Encontrar Adyacentes')
        print(state_val, comment)

def darTodosLosNodos(estructura, tipo):
    """
    Retorna una lista con todos los nodos de la lista

    Args:
        estructura: lista enlazada
        tipo: tipo de la lista (1:sencilla, 2:doble)
    Returns:
        -
    Raises:
        Exception
    """
    try:
        nodos = estructura.getNodeValues()
    except:
        raise Exception('\tProblema al obtener todos los elemento, método getNodeValues()')
    out.clear_output()
    displayList(estructura, tipo)
    txt = ''
    for i in nodos:
        txt = txt + str(i) + ', '
    
    print('Encontrar Todos')
    print('\tTotal Nodos:', str(len(nodos)))
    print('\tValores:', txt[:-2])


### Enlace - Árboles BST

In [12]:
def crearBST(init, file, data={}):
    """
    Crea un arbol BST

    Args:
        init: Vacia, Random, Estática, Archivo
        file: Estructura de datos externa
        data: JSON con la información de la inicializacion de la estructura

    Returns:
        La estructura de datos creada
    Raises:
        Exception
    """
    try:
        estructura = file.bst()
    except:
        e = "\tProblema al crear el arbol BST, método bst()"
        raise Exception(e)
    long = 0
    nodos = list()
    if init == 'Random':
        long = random.randint(5,10)
        nodos = create_n_random(long)
        for i in nodos:
            try:
                estructura.addNode_byValue(i)
            except:
                e = '\tProblema al añadir el elemento "'+str(i)+'", método addNode_byValue()'
                raise Exception(e)
    elif init == 'Estática' or init == 'Archivo':
        try:
            nodos = data["valores"]
        except:
            raise Exception("El formato del archivo ingresado no es válido")
        for i in nodos:
            try:
                estructura.addNode_byValue(i)
            except:
                e = '\tProblema al añadir el elemento "'+str(i)+'", método addNode_byValue()'
                raise Exception(e)
    displayBST(estructura) 
    
    try:
        st_nodos = estructura.getNodeValues("Preorder")
    except:
        e = '\tProblema al obtener todos los nodos, método getNodeValues()'
        raise Exception(e)
    state_val, comment = validar_bst_crear(nodos, st_nodos)
    
    print('Crear BST ' + init + ':')
    print(state_val)    
    print(comment)
    return estructura

def anadirNodoBST(estructura, nodo):
    """
    Añade un nodo al BST

    Args:
        estructura: lista enlazada
        nodo: valor del nodo
    Returns:
        -
    Raises:
        Exception
    """
    try:
        init_test = estructura.getNodeValues("Preorder")
    except:
        e = '\tProblema al obtener todos los nodos, método getNodeValues()'
        raise Exception(e)
    try:
        estructura.addNode_byValue(nodo)
        end_test = estructura.getNodeValues("Preorder")
        state_val, end_val, comment = validar_bst_anadir(init_test, end_test, nodo) 
    except:
        e = '\tProblema al añadir el elemento "'+str(nodo)+'", método addNode_byValue()'
        raise Exception(e)
    
    out.clear_output()
    displayBST(estructura)
    print('Añadir elemento')
    
    print(state_val + ':', comment)
    if state_val != VALIDATION_STATES[1]:
        print('Se esperaba:', end_val)
        print('Se obtuvo:  ', end_test)

def eliminarNodoBST(estructura, nodo):
    """
    Elimina un nodo al BST

    Args:
        estructura: BST
        nodo: valor del nodo
    Returns:
        -
    Raises:
        Exception
    """
    try:
        init_test = estructura.getNodeValues("Preorder")
    except:
        e = '\tProblema al obtener todos los nodos, método getNodeValues()'
        raise Exception(e)
    try:
        ans = estructura.deleteNode_byValue(nodo)
        end_test = estructura.getNodeValues("Preorder")
        state_val, end_val, comment = validar_bst_eliminar(init_test, end_test, nodo, ans) 
    except:
        e = '\tProblema al eliminar el elemento "'+str(nodo)+'", método deleteNode_byValue()'
    out.clear_output()
    displayBST(estructura)
    print('Eliminar elemento')
    
    print(state_val + ':', comment)
    if state_val == VALIDATION_STATES[-1]:
        print('Se esperaba:', end_val)
        print('Se obtuvo:  ', end_test)

def encontrarNodoBST(estructura, nodo):
    """
    Encuentra un nodo en la estrcutura

    Args:
        estructura: BST
        nodo: valor del nodo
    Returns:
        -
    Raises:
        Exception
    """
    try:
        ans = estructura.isNodeValue(nodo)
        nodos = estructura.getNodeValues("Preorder")
        state_val, ans_val, comment = validar_bst_encontrar(nodos, nodo, ans) 
    except:
        e = '\tProblema al buscar el elemento "' + str(nodo) + '", método isNodeValue()'
        raise Exception(e)
    out.clear_output()
    lista = list()
    lista.append(nodo)
    if ans_val:
        displayBST(estructura, lista)
    else:    
        displayBST(estructura)
    print('Existe elemento')
    print(state_val + ':', comment)
    
def findAdjacentNodoBST(estructura, nodo):
    """
    Encuentra los adyacentes de un nodo en el bst

    Args:
        estructura: lista enlazada
        nodo: valor del nodo
    Returns:
        -
    Raises:
        Exception
    """
    try:
        init_test = estructura.getNodeValues()
    except:
        e = '\tProblema al obtener todos los nodos, método getNodeValues()'
        raise Exception(e)
    try:
        listaAdj = estructura.findAdjacentNode(nodo)
        state_val, listaAdj_val, comment, exists = validar_bst_adyacentes(init_test, listaAdj, nodo) 
    except:
        e = '\tProblema al buscar los adyacentes del elemento "' + str(nodo) + '" , método findAdjacentNode()'
        raise Exception(e)
    
    out.clear_output()
    displayBST(estructura, listaAdj)
    
    if not exists:
        print('Encontrar Adyacentes\n\tEl elemento "'+str(nodo)+ '" no existe en el bst')
    else:
        print('Encontrar Adyacentes')
        print(state_val, comment)

def listarNodosBST(estructura, orden):
    """
    Lista todos los nodos del BST en el orden especificado

    Args:
        estructura: lista enlazada
        orden: orden en el cual se dan los nodos (preorden,inorden, postorden)
    Returns:
        -
    Raises:
        Exception
    """
    try:
        init_test = estructura.getNodeValues()
    except:
        e = '\tProblema al obtener todos los nodos, método getNodeValues()'
        raise Exception(e)
    try:
        nodos = estructura.getNodeValues(orden)
        state_val, nodos_val, comment = validar_bst_darNodos(init_test, nodos, orden) 
    except:
        e = '\tProblema al listar todos los elementos, método getNodeValues()'
        raise Exception(e)
    
    out.clear_output()
    displayBST(estructura)
    print('Listar todos los nodos')
    print(state_val, comment)


### Enlace - Grafos

In [13]:
def crearGraph(init, tipo, file=None, data={},labels=False):
    """
    Crea un grafo

    Args:
        init: tipo de inicializacion de la estructura
        tipo: Dirigido(4) o no Dirigido(5)
        file: estructura de datos externa
        data: JSON con información de inicializacion
        labels: si se muestran o no los labels de los pesos
    Returns:
        La estructura creada
    Raises:
        Exception
    """
    
    if tipo == 4: txt = 'Dirigido - ' + init
    else: txt = 'No Dirigido - ' + init
    
    txtNodos = ''
    txtEdges = ''
    try:
        if tipo == 4:
            estructura = file.grafo('Directed')
        else:
            estructura = file.grafo()
    except:
        e = "\tProblema al crear grafo, método graph()"
        raise Exception(e)
    long = 0
    nodos = list()
    edges = list()
    if init == 'Random':
        long = random.randint(10,15)
        nodos = create_n_random(long)
        edges = create_n_randomEdges(nodos, tipo)
        
        txtNodos = 'Nodos ('+str(len(nodos))+'): '
        if tipo == 5:
            txtEdges = 'Arcos ('+str(len(edges)*2)+'):\n'
        else:
            txtEdges = 'Arcos ('+str(len(edges))+'):\n'
        for i in nodos:
            try:
                estructura.addNode_byValue(str(i))
                txtNodos = txtNodos + str(i) + ', '
            except:
                e = '\tProblema al añadir el elemento "'+str(i)+'", método addNode_byValue()'
                raise Exception(e)
        for i,j,k in edges:
            try:
                estructura.addEdge_byValue(i,j,k)
                if tipo == 5:
                    arco = '(' + str(i) + ' <-> ' + str(j) + ', ' + str(round(k,2))+ ')'
                else:
                    arco = '(' + str(i) + ' -> ' + str(j) + ', ' + str(round(k,2))+ ')'
                txtEdges = txtEdges + '\t' + arco + '\n'
            except:
                e = '\tProblema al añadir el arco "('+str(i)+ '->'+ str(j)+','+str(k) + ')", método addEdge_byValue()'
                raise Exception(e)
        
    elif init == 'Estática' or init == 'Archivo':
        try:
            nodos = data["nodos"]
            edges = data["edges"]
            txtNodos = 'Nodos ('+str(len(nodos))+'): '
            if tipo == 5:
                txtEdges = 'Arcos ('+str(len(edges)*2)+'):\n'
            else:
                txtEdges = 'Arcos ('+str(len(edges))+'):\n'
                
            for i in nodos:
                try:
                    estructura.addNode_byValue(str(i))
                    txtNodos = txtNodos + str(i) + ', '
                except:
                    e = '\tProblema al añadir el elemento "'+str(i)+'", método addNode_byValue()'
                    raise Exception(e)
            for edge in edges:
                try:
                    estructura.addEdge_byValue(str(edge[0]),str(edge[1]),edge[2])
                    if tipo == 5:
                        arco = '(' + str(edge[0]) + ' <-> ' + str(edge[1]) + ', ' + str(edge[2])+ ')'
                    else:
                        arco = '(' + str(edge[0]) + ' -> ' + str(edge[1]) + ', ' + str(edge[2])+ ')'
                    txtEdges = txtEdges + '\t' + arco + '\n'
                except:
                    e = '\tProblema al añadir el arco "('+str(edge[0])+ '->'+ str(edge[1])+','+str(edge[2]) + ')", método addEdge_byValue()'
                    raise Exception(e)
        except:
            raise Exception("El formato del archivo ingresado no es válido")
    displayGraph(estructura, tipo, label=labels)
     
    try:
        st_nodos = estructura.getNodeValues()
    except:
        e = '\tProblema al obtener todos los nodos, método getNodeValues()'
        raise Exception(e)
    try:
        st_edges = estructura.getEdgeValues()
    except:
        e = '\tProblema al obtener todos los nodos, método getEdgeValues()'
        raise Exception(e)    
    
    state_val, comment = validar_graph_crear(nodos, edges, st_nodos, st_edges, tipo)
    
    print('Crear Grafo', txt)
    print(txtNodos[:-2])
    print(txtEdges)
    print(state_val)
    print(comment)
    return estructura

def anadirNodoGraph(estructura, tipo, label, nodo):
    """
    Añade un nodo al grafo

    Args:
        estructura: grafo
        tipo: Dirigido(4) o no Dirigido(5)
        label: si se muestran o no los labels de los pesos
        nodo: valor del nodo

    Returns:
        -
    Raises:
        Exception
    """
    try:
        init_test = estructura.getNodeValues()
    except:
        e = '\tProblema al obtener todos los nodos, método getNodeValues()'
        raise Exception(e)
    try:
        estructura.addNode_byValue(nodo)
        end_test = estructura.getNodeValues()
        state_val, comment = validar_graph_anadir(init_test, end_test, tipo, nodo) 
    except:
        e = '\tProblema al añadir el elemento "'+str(nodo)+'", método addNode_byValue()'
        raise Exception(e)
    
    out.clear_output()
    displayGraph(estructura, tipo, label, nodosX=[nodo])
    print('Añadir nodo')
    print(state_val, comment)

def eliminarNodoGraph(estructura, tipo, label, nodo):
    """
    Elimina un nodo del grafo

    Args:
        estructura: grafo
        tipo: Dirigido(4) o no Dirigido(5)
        label: si se muestran o no los labels de los pesos
        nodo: valor del nodo
    Returns:
        -
    Raises:
        Exception
    """
    try:
        init_test = estructura.getNodeValues()
    except:
        e = '\tProblema al obtener todos los nodos, método getNodeValues()'
        raise Exception(e)
    try:
        estructura.deleteNode_byValue(nodo)
        end_test = estructura.getNodeValues()
        state_val, comment = validar_graph_eliminar(init_test, end_test, tipo, nodo) 
    except:
        e = '\tProblema al eliminar el elemento "'+str(nodo)+'", método deleteNode_byValue()'
        raise Exception(e)
    
    out.clear_output()
    displayGraph(estructura, tipo, label)
    print('Eliminar nodo')
    print(state_val, comment)
    
def existeNodoGraph(estructura, tipo, label, nodo):
    """
    Verifica la existencia de un nodo en el grafo

    Args:
        estructura: grafo
        tipo: Dirigido(4) o no Dirigido(5)
        label: si se muestran o no los labels de los pesos
        nodo: valor del nodo
    Returns:
        -
    Raises:
        Exception
    """
    try:
        init_test = estructura.getNodeValues()
    except:
        e = '\tProblema al obtener todos los nodos, método getNodeValues()'
        raise Exception(e)
    try:
        existe_test = estructura.isNodeValue(nodo)
        state_val, comment = validar_graph_existe(init_test, existe_test, nodo) 
    except:
        e = '\tProblema al verificar si existe el nodo "'+str(nodo)+'", método isNodeValue()'
        raise Exception(e)
    
    out.clear_output()
    displayGraph(estructura, tipo, label, nodosX=[nodo])
    print('Existe nodo')
    print(state_val, comment)
    
def anadirArcoGraph(estructura, tipo, label, origen, destino, peso):
    """
    Añade un arco al grafo

    Args:
        estructura: grafo
        tipo: Dirigido(4) o no Dirigido(5)
        label: si se muestran o no los labels de los pesos
        origen: valor del nodo origen
        destino: valor del nodo destino
        peso: peso del arco entre origen y destino
    Returns:
        -
    Raises:
        Exception
    """
    out.clear_output()
    displayGraph(estructura, tipo, label)
    try:
        nodes = estructura.getNodeValues()
    except:
        e = '\tProblema al obtener todos los nodos, método getNodeValues()'
        raise Exception(e)
    try:
        init_test = estructura.getEdgeValues()
    except:
        e = '\tProblema al obtener todos los arcos, método getEdgeValues()'
        raise Exception(e)
    try:
        estructura.addEdge_byValue(origen,destino,peso)
    except:
        arco = '(' + str(origen) + ' -> ' + str(destino) + ',' + str(peso) + ')'
        e = '\tProblema al añadir el arco "'+arco+'", método addEdge_byValue()\n\tVerificar la existencia de los vertices'
        raise Exception(e)
    try:
        end_test = estructura.getEdgeValues()
        state_val, comment = validar_graph_anadirEdge(nodes, init_test, end_test, tipo, origen, destino, peso) 
    except:
        e = '\tProblema en el metodo de validacion'
        raise Exception(e)
    out.clear_output()
    displayGraph(estructura, tipo, label, edgesX=[(origen,destino)])
    print('Añadir Arco')
    print(state_val, comment)
    
def adyacentesNodoGraph(estructura, tipo, label, nodo):
    """
    Encuentra los adyacentes de un nodo en el grafo

    Args:
        estructura: grafo
        tipo: Dirigido(4) o no Dirigido(5)
        label: si se muestran o no los labels de los pesos
        nodo: valor del nodo
    Returns:
        -
    Raises:
        Exception
    """
    try:
        nodes = estructura.getNodeValues()
    except:
        e = '\tProblema al obtener todos los nodos, método getNodeValues()'
        raise Exception(e)
    try:
        edges = estructura.getEdgeValues()
    except:
        e = '\tProblema al obtener todos los arcos, método getEdgeValues()'
        raise Exception(e)
    try:
        adjNodes = estructura.findAdjacentNode(nodo)
        state_val, comment = validar_graph_adj(nodes, edges, adjNodes, nodo, tipo) 
    except:
        e = '\tProblema al encontrar los adyacentes del nodo "'+str(nodo)+'", método findAdjacentNode()'
        raise Exception(e)
    
    paintEdges = list()
    for i in adjNodes:
        paintEdges.append((nodo,i))
    
    out.clear_output()
    displayGraph(estructura, tipo, label, nodosX=adjNodes, edgesX=paintEdges)
    print('Adyacentes nodo')
    print(state_val, comment)    

def encontrarNodosGraph(estructura, tipo, label):
    """
    Encuentra todos los nodos del grafo

    Args:
        estructura: grafo
        tipo: Dirigido(4) o no Dirigido(5)
        label: si se muestran o no los labels de los pesos
    Returns:
        -
    Raises:
        Exception
    """
    try:
        nodes = estructura.getNodeValues()
    except:
        e = '\tProblema al obtener todos los nodos, método getNodeValues()'
        raise Exception(e)

    state_val, comment = validar_graph_todos(nodes, tipo)

    out.clear_output()
    displayGraph(estructura, tipo, label)
    print('Encontrar todos los nodos:', state_val)
    print(comment)

def recorridosGraph(estructura, tipo, label, recorrido, nodo=None):    
    """
    Ejecuta un algoritmo dado en el grafo

    Args:
        estructura: grafo
        tipo: Dirigido(4) o no Dirigido(5)
        label: si se muestran o no los labels de los pesos
        recorrido: nombre del algoritmo que se va a ejecutar
        nodo: nodo de inicio del algoritmo

    Returns:
        -
    Raises:
        Exception
    """
    if nodo is not None:
        try:
            out.clear_output()
            displayGraph(estructura, tipo, label)
            existe = estructura.isNodeValue(nodo)
        except:
            e = '\tProblema al verificar la existencia del nodo '+ nodo +', en el metodo getNodeValues()'
            raise Exception(e)
        if not existe:
            e = '\tEl nodo "'+ nodo +'" no pertenece al grafo.'
            raise Exception(e)
    rt, cmm = checkAlgoritmGraph(tipo, recorrido)
    if not rt:
        out.clear_output()
        displayGraph(estructura, tipo, label)
        raise Exception(cmm)
    try:
        rtaRecorrido = estructura.algorithms(recorrido, nodo)
        print(rtaRecorrido)
    except:
        e = '\tProblema al ejecutar el recorrido ' + recorrido + ', metodo algorithms()'
        raise Exception(e)

    out.clear_output()
    state_val = ''
    comment = ''
    if recorrido == 'DepthFirstSearch' or recorrido == 'BreadhtFirstSearch' or recorrido == 'DepthFirstOrder':
        state_val, comment = validarRecorridosGrafo(estructura, tipo, 'lista_nodos', recorrido, rtaRecorrido, nodo)
        if recorrido != 'DepthFirstOrder':
            rtaRecorrido.remove(nodo)
            displayGraph(estructura, tipo, label, nodosX=rtaRecorrido, nodeY=nodo)
        else:
            displayGraph(estructura, tipo, label)
            
    elif recorrido == 'DirectedCycle':
        nodes = getNodesGivenEdges(rtaRecorrido)
        state_val, comment = validarRecorridosGrafo(estructura, tipo, 'edges', recorrido, rtaRecorrido)
        displayGraph(estructura, tipo, label, nodosX=nodes,edgesX=rtaRecorrido)  
        
    elif recorrido == 'Dijkstra' or recorrido == 'Bellman-Ford':
        state_val, comment = validarRecorridosGrafo(estructura, tipo, 'dicts', recorrido, rtaRecorrido, nodo)
        displayGraph(estructura, tipo, label)
        
    elif recorrido == 'KosarajuSCC':
        state_val, comment = validarRecorridosGrafo(estructura, tipo, 'single_dict', recorrido, rtaRecorrido)
        displayGraph(estructura, tipo, label)
        
    elif recorrido == 'PrimMST':
        state_val, comment = validarRecorridosGrafo(estructura, tipo, 'tupla', recorrido, rtaRecorrido, nodo)
        edges = rtaRecorrido[0]
        nodes = getNodesGivenEdges(edges)
        displayGraph(estructura, tipo, label, nodosX=nodes,edgesX=edges)
    print('Algoritmo ' + recorrido + ':' , state_val)
    print(comment)


## 5. Interfaz Grafica

### Opciones

En esta seccion se construyen los paneles de las opciones de las operaciones que se soportan por cada estructura de datos. 

Es acá donde se hace el llamado a los metodos de Enlace segun corresponda

#### Opciones - Listas Enlazadas

In [14]:
import this

listaType = widgets.RadioButtons(
    options = ['Sencilla', 'Doble'],
    disabled = False
)

grafoType = widgets.RadioButtons(
    options = ['Dirigido', 'No dirigido'],
    disabled = False
)

grafoLabels = widgets.Checkbox(
    value=False,
    description='mostrar pesos',
    disabled=False,
    indent=False
)

listaInit = widgets.RadioButtons(
    options = ['Vacia', 'Random', 'Estática' ,'Archivo'],
    disabled = False
)

listaOrden = widgets.RadioButtons(
    options = ['Preorder', 'Inorder', 'Postorder'],
    disabled = False
)

listaInputNode = widgets.Text(
    value = '',
    placeholder = 'Valor del nodo',
    disabled = False,
    layout = {'width': '140px', 'justify_content':'center'}
)

btn_crear = Button(description='Crear', width='extended')
def on_button_crear_clicked(b):
    with out:
        out.clear_output()
        if listaType.value == 'Sencilla':  this.type = 1
        else: this.type = 2
        try:
            if listaInit.value == 'Estática':
                try:
                    name = 'creation files/lista_enlazada_1.json'
                    json_data = open(name)
                    data = json.load(json_data)
                except:
                    e = "\tProblema al cargar el archivo " + name
                    raise Exception (e)               
                this.estructura = crearListaEnlazada(this.type, this.file, listaInit.value, data)
            elif listaInit.value == 'Archivo':
                try:
                    root = tk.Tk()
                    file_path = tk.filedialog.askopenfilename(title='Select a file...',filetypes=[("Json File", "*.json")])
                    root.destroy()
                    json_data = open(file_path)
                    data = json.load(json_data)
                except:
                    e = "\tProblema al cargar el archivo " + file_path
                    raise Exception (e)
                this.estructura = crearListaEnlazada(this.type, this.file, listaInit.value, data)
            else:
                this.estructura = crearListaEnlazada(this.type, this.file, listaInit.value)
        except Exception as e:
            print('Hubo un problema al intentar crear la lista')
            print(e)
btn_crear.on_click(on_button_crear_clicked)

btn_addNode = Button(description='Añadir', width='extended')
def on_button_addNode_clicked(b):
    with out:
        value = listaInputNode.value.strip() 
        if len(value) > 0:
            try:
                try:
                    data = int(value)
                except:
                    data = value
                try:
                    this.estructura
                    state, comment = validarEstructura([1,2], this.type)
                    if not state:
                        print(comment)
                except:
                    raise Exception("\tLa estructura no ha sido creada")
                if state:
                    anadirNodoLista(this.estructura, this.type, data)
                    listaInputNode.value = ''
            except Exception as e:
                print('Hubo un problema al intentar añadir un elemento')
                print(e)
btn_addNode.on_click(on_button_addNode_clicked)

btn_deleteNode = Button(description='Eliminar', width='extended')
def on_button_deleteNode_clicked(b):
    with out:
        value = listaInputNode.value.strip() 
        if len(value) > 0:
            try:
                try:
                    data = int(value)
                except:
                    data = value
                try:
                    this.estructura
                    state, comment = validarEstructura([1,2], this.type)
                    if not state:
                        print(comment)
                except:
                    raise Exception("\tLa estructura no ha sido creada")
                if state:
                    eliminarNodoLista(this.estructura, this.type, data)
                    listaInputNode.value = ''
            except Exception as e:
                print('Hubo un problema al intentar eliminar un elemento')
                print(e)
btn_deleteNode.on_click(on_button_deleteNode_clicked)

btn_findNode = Button(description='Validar', width='extended')
def on_button_findNode_clicked(b):
    with out:
        value = listaInputNode.value.strip() 
        if len(value) > 0:
            try:
                try:
                    data = int(value)
                except:
                    data = value
                try:
                    this.estructura
                    state, comment = validarEstructura([1,2], this.type)
                    if not state:
                        print(comment)
                except:
                    raise Exception("\tLa estructura no ha sido creada")
                if state:                
                    encontrarNodoLista(this.estructura, this.type, data)
                    listaInputNode.value = ''
            except Exception as e:
                print('Hubo un problema al intentar encontrar un elemento')
                print(e)
btn_findNode.on_click(on_button_findNode_clicked)

btn_findAdjNode = Button(description='Adyacentes', width='extended')
def on_button_findAdjNode_clicked(b):
    with out:
        value = listaInputNode.value.strip() 
        if len(value) > 0:
            try:
                try:
                    data = int(value)
                except:
                    data = value
                try:
                    this.estructura
                    state, comment = validarEstructura([1,2], this.type)
                    if not state:
                        print(comment)
                except:
                    raise Exception("\tLa estructura no ha sido creada")
                if state:
                    findAdjacentNodeLista(this.estructura, this.type, data)
                    listaInputNode.value = ''
            except Exception as e:
                print('Hubo un problema al intentar encontrar los adyacentes de un elemento')
                print(e)
btn_findAdjNode.on_click(on_button_findAdjNode_clicked)

btn_todosNodos = Button(description='Encontrar Todos', width='extended')
def on_button_todosNodos_clicked(b):
    with out:    
        try:
            try:
                this.estructura
                state, comment = validarEstructura([1,2], this.type)
                if not state:
                    print(comment)
            except:
                raise Exception("\tLa estructura no ha sido creada")
            if state:
                darTodosLosNodos(this.estructura, this.type)
        except Exception as e:
            print('Hubo un problema al intentar encontrar todos los nodos')
            print(e)
btn_todosNodos.on_click(on_button_todosNodos_clicked)

The Zen of Python, by Tim Peters

Beautiful is better than ugly.
Explicit is better than implicit.
Simple is better than complex.
Complex is better than complicated.
Flat is better than nested.
Sparse is better than dense.
Readability counts.
Special cases aren't special enough to break the rules.
Although practicality beats purity.
Errors should never pass silently.
Unless explicitly silenced.
In the face of ambiguity, refuse the temptation to guess.
There should be one-- and preferably only one --obvious way to do it.
Although that way may not be obvious at first unless you're Dutch.
Now is better than never.
Although never is often better than *right* now.
If the implementation is hard to explain, it's a bad idea.
If the implementation is easy to explain, it may be a good idea.
Namespaces are one honking great idea -- let's do more of those!


#### Opciones - Árboles BST 

In [15]:
btn_crearArbolOption = Button(description='Crear', width='extended')
def on_button_crearArbolOption_clicked(b):
    with out:
        out.clear_output()
        try:
            this.type = 3
            if listaInit.value == 'Estática':
                try:
                    name = 'creation files/bst_1.json' 
                    json_data = open(name)
                    data = json.load(json_data)
                except:
                    e = "\tProblema al cargar el archivo " + name
                    raise Exception (e)               
                this.estructura = crearBST(listaInit.value, this.file, data)
            elif listaInit.value == 'Archivo':
                try:
                    root = tk.Tk()
                    file_path = tk.filedialog.askopenfilename(title='Select a file...',filetypes=[("Json File", "*.json")])
                    root.destroy()
                    json_data = open(file_path)
                    data = json.load(json_data)
                except:
                    e = "\tProblema al cargar el archivo " + file_path
                    raise Exception (e)
                this.estructura = crearBST(listaInit.value, this.file, data)
            else:
                this.estructura = crearBST(listaInit.value, this.file)
        except Exception as e:
            print('Hubo un problema al intentar crear la lista')
            print(e)
btn_crearArbolOption.on_click(on_button_crearArbolOption_clicked)

btn_addNodeArbolOption = Button(description='Añadir', width='extended')
def on_button_addNodeArbolOption_clicked(b):
    with out:
        value = listaInputNode.value.strip() 
        if len(value) > 0:
            try:
                try:
                    data = int(value)
                except:
                    data = value
                try:
                    this.estructura
                    state, comment = validarEstructura([3], this.type)
                    if not state:
                        print(comment)
                except:
                    raise Exception("\tLa estructura no ha sido creada")
                if state:
                    anadirNodoBST(this.estructura, data)
                    listaInputNode.value = ''
            except Exception as e:
                print('Hubo un problema al intentar añadir un elemento')
                print(e)
btn_addNodeArbolOption.on_click(on_button_addNodeArbolOption_clicked)

btn_deleteNodeArbolOption = Button(description='Eliminar', width='extended')
def on_button_deleteNodeArbolOption_clicked(b):
    with out:
        value = listaInputNode.value.strip() 
        if len(value) > 0:
            try:
                try:
                    data = int(value)
                except:
                    data = value
                try:
                    this.estructura
                    state, comment = validarEstructura([3], this.type)
                    if not state:
                        print(comment)
                except:
                    raise Exception("\tLa estructura no ha sido creada")
                if state:
                    eliminarNodoBST(this.estructura, data)
                    listaInputNode.value = ''
            except Exception as e:
                print('Hubo un problema al intentar eliminar un elemento')
                print(e)
btn_deleteNodeArbolOption.on_click(on_button_deleteNodeArbolOption_clicked)

btn_findNodeArbolOption = Button(description='Validar', width='extended')
def on_button_findNodeArbolOption_clicked(b):
    with out:
        value = listaInputNode.value.strip() 
        if len(value) > 0:
            try:
                try:
                    data = int(value)
                except:
                    data = value
                try:
                    this.estructura
                    state, comment = validarEstructura([3], this.type)
                    if not state:
                        print(comment)
                except:
                    raise Exception("\tLa estructura no ha sido creada")
                if state:
                    encontrarNodoBST(this.estructura, data)
                    listaInputNode.value = ''
            except Exception as e:
                print('Hubo un problema al intentar encontrar un elemento')
                print(e)
btn_findNodeArbolOption.on_click(on_button_findNodeArbolOption_clicked)

btn_findAdjNodeArbolOption = Button(description='Adyacentes', width='extended')
def on_button_findAdjNodeArbolOption_clicked(b):
    with out:
        value = listaInputNode.value.strip() 
        if len(value) > 0:
            try:
                try:
                    data = int(value)
                except:
                    data = value
                try:
                    this.estructura
                    state, comment = validarEstructura([3], this.type)
                    if not state:
                        print(comment)
                except:
                    raise Exception("\tLa estructura no ha sido creada")
                if state:
                    findAdjacentNodoBST(this.estructura, data)
                    listaInputNode.value = ''
            except Exception as e:
                print('Hubo un problema al intentar encontrar los adyacentes de un elemento')
                print(e)
btn_findAdjNodeArbolOption.on_click(on_button_findAdjNodeArbolOption_clicked)

btn_todosNodosArbol = Button(description='Listar nodos', width='extended')
def on_button_todosNodosArbol_clicked(b):
    with out:
        out.clear_output()
        try:
            try:
                this.estructura
                state, comment = validarEstructura([3], this.type)
                if not state:
                    print(comment)
            except:
                raise Exception("\tLa estructura no ha sido creada")
            if state:
                listarNodosBST(this.estructura, listaOrden.value)
        except Exception as e:
            print('Hubo un problema al intentar encontrar los adyacentes de un elemento')
            print(e)
btn_todosNodosArbol.on_click(on_button_todosNodosArbol_clicked)

#### Opciones - Grafos

In [16]:
grafoInputNodeInit = widgets.Text(
    value = '',
    placeholder = 'Nodo Origen',
    disabled = False,
    layout = {'width': '140px', 'justify_content':'center'}
)
grafoInputNodeEnd = widgets.Text(
    value = '',
    placeholder = 'Nodo Destino',
    disabled = False,
    layout = {'width': '140px', 'justify_content':'center'}
)
grafoInputEdgeWeight = widgets.FloatText(
    step=0.5,
    value = 0.00,
    placeholder = 'Peso Arco',
    disabled = False,
    layout = {'width': '140px', 'justify_content':'center'}
)
grafoInputNode = widgets.Text(
    value = '',
    placeholder = 'Vertice de inicio *',
    disabled = False,
    layout = {'width': '140px', 'justify_content':'center'}
)
grafoRecorrido = widgets.RadioButtons(
    options = ['DepthFirstSearch*', 'BreadhtFirstSearch*', 'DepthFirstOrder', 
               'PrimMST*', 'KosarajuSCC', 'Dijkstra*', 'DirectedCycle', 'Bellman-Ford*' ],
    disabled = False
)
btn_crearGrafoOption = Button(description='Crear', width='extended')
def on_button_crearGrafoOption_clicked(b):
    with out:
        out.clear_output()
        if grafoType.value == 'Dirigido':  this.type = 4
        else: this.type = 5
        try:
            if listaInit.value == 'Estática':
                try:
                    name = 'creation files/graph_1.json' 
                    json_data = open(name)
                    data = json.load(json_data)
                except:
                    e = "\tProblema al cargar el archivo " + name
                    raise Exception (e)               
                this.estructura = crearGraph(init = listaInit.value, 
                                             tipo = this.type, 
                                             data = data, 
                                             file = this.file,
                                             labels=grafoLabels.value)
            elif listaInit.value == 'Archivo':
                try:
                    root = tk.Tk()
                    file_path = tk.filedialog.askopenfilename(title='Select a file...',filetypes=[("Json File", "*.json")])
                    root.destroy()
                    json_data = open(file_path)
                    data = json.load(json_data)
                except:
                    e = "\tProblema al cargar el archivo " + file_path
                    raise Exception (e)
                this.estructura = crearGraph(init = listaInit.value, 
                                tipo = this.type, 
                                data = data, 
                                file = this.file,
                                labels=grafoLabels.value)
            else:
                this.estructura = crearGraph(init = listaInit.value, 
                                tipo = this.type,
                                file = this.file, 
                                labels=grafoLabels.value)
        except Exception as e:
            print('Hubo un problema al intentar crear la lista')
            print(e)
btn_crearGrafoOption.on_click(on_button_crearGrafoOption_clicked)

btn_addNodeGrafoOption = Button(description='Añadir', width='extended')
def on_button_addNodeGrafoOption_clicked(b):
    with out:
        value = listaInputNode.value.strip() 
        if len(value) > 0:
            try:
                try:
                    this.estructura
                    state, comment = validarEstructura([4,5], this.type)
                    if not state:
                        print(comment)
                except:
                    raise Exception("\tLa estructura no ha sido creada")
                if state:
                    anadirNodoGraph(this.estructura, this.type, grafoLabels.value, value)
                    listaInputNode.value = ''
            except Exception as e:
                print('Hubo un problema al intentar añadir un nodo')
                print(e)
btn_addNodeGrafoOption.on_click(on_button_addNodeGrafoOption_clicked)

btn_deleteNodeGrafoOption = Button(description='Eliminar', width='extended')
def on_button_deleteNodeGrafoOption_clicked(b):
    with out:
        value = listaInputNode.value.strip() 
        if len(value) > 0:
            try:
                try:
                    this.estructura
                    state, comment = validarEstructura([4,5], this.type)
                    if not state:
                        print(comment)
                except:
                    raise Exception("\tLa estructura no ha sido creada")
                if state:
                    eliminarNodoGraph(this.estructura, this.type, grafoLabels.value, value)
                    listaInputNode.value = ''
            except Exception as e:
                print('Hubo un problema al intentar eliminar un nodo')
                print(e)
btn_deleteNodeGrafoOption.on_click(on_button_deleteNodeGrafoOption_clicked)

btn_findNodeGrafoOption = Button(description='Validar', width='extended')
def on_button_findNodeGrafoOption_clicked(b):
    with out:
        value = listaInputNode.value.strip() 
        if len(value) > 0:
            try:
                try:
                    this.estructura
                    state, comment = validarEstructura([4,5], this.type)
                    if not state:
                        print(comment)
                except:
                    raise Exception("\tLa estructura no ha sido creada")
                if state:
                    existeNodoGraph(this.estructura, this.type, grafoLabels.value, value)
                    listaInputNode.value = ''
            except Exception as e:
                print('Hubo un problema al intentar validar si un nodo existe')
                print(e)
btn_findNodeGrafoOption.on_click(on_button_findNodeGrafoOption_clicked)

btn_addEdgeOption = Button(description='Añadir', width='extended')
def on_button_addEdgeOption_clicked(b):
    with out:
        origen = grafoInputNodeInit.value.strip() 
        destino = grafoInputNodeEnd.value.strip()
        peso = grafoInputEdgeWeight.value
        if len(origen) > 0 and len(destino) > 0:
            try:
                try:
                    this.estructura
                    state, comment = validarEstructura([4,5], this.type)
                    if not state:
                        print(comment)
                except:
                    raise Exception("\tLa estructura no ha sido creada")
                if state:
                    anadirArcoGraph(this.estructura, this.type, grafoLabels.value, origen, destino, peso)
                grafoInputNodeInit.value = ''
                grafoInputNodeEnd.value = ''
                grafoInputEdgeWeight.value = 0.00
            except Exception as e:
                print('Hubo un problema al intentar añadir un arco')
                print(e)
btn_addEdgeOption.on_click(on_button_addEdgeOption_clicked)

btn_findAdjNodeGrafoOption = Button(description='Adyacentes', width='extended')
def on_button_findAdjNodeGrafoOption_clicked(b):
    with out:
        value = listaInputNode.value.strip() 
        if len(value) > 0:
            try:
                try:
                    this.estructura
                    state, comment = validarEstructura([4,5], this.type)
                    if not state:
                        print(comment)
                except:
                    raise Exception("\tLa estructura no ha sido creada")
                if state:
                    adyacentesNodoGraph(this.estructura, this.type, grafoLabels.value, value)
                listaInputNode.value = ''
            except Exception as e:
                print('Hubo un problema al intentar encontrar los adyacentes de un nodo')
                print(e)
btn_findAdjNodeGrafoOption.on_click(on_button_findAdjNodeGrafoOption_clicked)

btn_todosNodosGrafo = Button(description='Encontrar Todos', width='extended')
def on_button_todosNodosGrafo_clicked(b):
    with out:
        try:
            try:
                this.estructura
                state, comment = validarEstructura([4,5], this.type)
                if not state:
                    print(comment)
            except:
                raise Exception("\tLa estructura no ha sido creada")
            if state:
                encontrarNodosGraph(this.estructura, this.type, grafoLabels.value)
        except Exception as e:
            print('Hubo un problema al intentar encontrar todos los nodos')
            print(e)
btn_todosNodosGrafo.on_click(on_button_todosNodosGrafo_clicked)

btn_recorridosGrafo = Button(description='Ejecutar', width='extended')
def on_button_recorridosGrafo_clicked(b):
    with out:
        try:
            try:
                this.estructura
                state, comment = validarEstructura([4,5], this.type)
                if not state:
                    print(comment)
            except:
                raise Exception("\tLa estructura no ha sido creada")
            if state:
                recorrido = grafoRecorrido.value
                nodo = grafoInputNode.value.strip()
                if '*' in recorrido and len(nodo) > 0:
                    recorrido = recorrido.replace('*', '')
                    recorridosGraph(this.estructura, this.type, grafoLabels.value, recorrido, nodo)
                elif '*' in recorrido:
                    recorrido = recorrido.replace('*', '')
                    out.clear_output()
                    displayGraph(this.estructura, this.type, grafoLabels.value)
                    raise Exception('ERROR: Para el recorrido ' + recorrido + ' debe ingresar un vertice')
                else:
                    recorridosGraph(this.estructura, this.type, grafoLabels.value, recorrido)
                grafoInputNode.value = ''
        except Exception as e:
            #out.clear_output()
            print('Hubo un problema al intentar ejecutar el recorrido en el grafo')
            print(e)
btn_recorridosGrafo.on_click(on_button_recorridosGrafo_clicked)

#### Opciones - Nuevo

In [17]:
# -------------------------- Cargar Estructura - Nueva --------------------------#
#btn_ejecutarNuevaOpcion = Button(description='Ejecutar Nuevo', width='extended')
#def on_button_ejecutarNuevaOpcion_clicked(b):
#    with out:
#        '''
#        En esta parte se hace el llamado a los metodos de enlace, se validan los inputs, se grafica, etc.
#        '''
#        print('Nueva Opcion')
#btn_ejecutarNuevaOpcion.on_click(on_button_ejecutarNuevaOpcion_clicked)
# ------------------------------------------------------------------------------#

### Layouts

Sección en la que se definen los layout de los componentes (botones y opciones) para cada una de las estructuras de datos

#### Layout - Listas Enlazadas

In [18]:
items_optionsCreateList = [
    Box([Label(value=' ')]),
    Box([Label(value='Crear Lista Enlazada')], layout=form_item_layout),
    Box([Label(value='Tipo:')]),
    Box([listaType]),
    Box([Label(value='Inicialización:')]),
    Box([listaInit]),
    Box([btn_crear]),
]
items_optionsAddNode = [
    Box([Label(value=' ')]),
    Box([Label(value='Añadir Elemento')], layout=form_item_layout),
    Box([listaInputNode], layout=form_item_layout),
    Box([btn_addNode], layout=form_item_layout),
]
items_optionsDeleteNode = [
    Box([Label(value=' ')]),
    Box([Label(value='Eliminar Elemento')], layout=form_item_layout),
    Box([listaInputNode], layout=form_item_layout),
    Box([btn_deleteNode], layout=form_item_layout),
]
items_optionsFindNode = [
    Box([Label(value=' ')]),
    Box([Label(value='Existe Elemento')], layout=form_item_layout),
    Box([listaInputNode], layout=form_item_layout),
    Box([btn_findNode], layout=form_item_layout),
]
items_optionsFindAdjNode = [
    Box([Label(value=' ')]),
    Box([Label(value='Adyacentes Elemento')], layout=form_item_layout),
    Box([listaInputNode], layout=form_item_layout),
    Box([btn_findAdjNode], layout=form_item_layout),
]
items_optionsFindAllNodes = [
    Box([Label(value=' ')]),
    Box([btn_todosNodos], layout=form_item_layout),
]

layoutOptionsCreate = Layout(
    display='none',
    flex_flow='column',
    align_items='stretch',
    width='auto',
    justify_content='center'
)
layoutOptionsAddNode = Layout(
    display='none',
    flex_flow='column',
    align_items='stretch',
    width='auto',
    justify_content='center'
)
layoutOptionsDeleteNode = Layout(
    display='none',
    flex_flow='column',
    align_items='stretch',
    width='auto',
    justify_content='center'
)
layoutOptionsFindNode = Layout(
    display='none',
    flex_flow='column',
    align_items='stretch',
    width='auto',
    justify_content='center'
)
layoutOptionsFindAdjNode = Layout(
    display='none',
    flex_flow='column',
    align_items='stretch',
    width='auto',
    justify_content='center'
)
layoutOptionsFindAllNodes = Layout(
    display='none',
    flex_flow='column',
    align_items='stretch',
    width='auto',
    justify_content='center'
)

optionsCreateList = Box(items_optionsCreateList, layout=layoutOptionsCreate)
optionsAddNode = Box(items_optionsAddNode, layout=layoutOptionsAddNode)
optionsDeleteNode = Box(items_optionsDeleteNode, layout=layoutOptionsDeleteNode)
optionsFindNode = Box(items_optionsFindNode, layout=layoutOptionsFindNode)
optionsFindAdjNode = Box(items_optionsFindAdjNode, layout=layoutOptionsFindAdjNode)
optionsFindAllNodes = Box(items_optionsFindAllNodes, layout=layoutOptionsFindAllNodes)


#### Layout - Árboles BST 

In [19]:
items_optionsCreateArbol = [
    Box([Label(value=' ')]),
    Box([Label(value='Crear Árbol Binario')], layout=form_item_layout),
    Box([Label(value='Inicialización:')]),
    Box([listaInit]),
    Box([btn_crearArbolOption]),
]
items_optionsAddNodeArbol = [
    Box([Label(value=' ')]),
    Box([Label(value='Añadir Elemento')], layout=form_item_layout),
    Box([listaInputNode], layout=form_item_layout),
    Box([btn_addNodeArbolOption], layout=form_item_layout),
]
items_optionsDeleteNodeArbol = [
    Box([Label(value=' ')]),
    Box([Label(value='Eliminar Elemento')], layout=form_item_layout),
    Box([listaInputNode], layout=form_item_layout),
    Box([btn_deleteNodeArbolOption], layout=form_item_layout),
]
items_optionsFindNodeArbol = [
    Box([Label(value=' ')]),
    Box([Label(value='Existe Elemento')], layout=form_item_layout),
    Box([listaInputNode], layout=form_item_layout),
    Box([btn_findNodeArbolOption], layout=form_item_layout),
]
items_optionsFindAdjNodeArbol = [
    Box([Label(value=' ')]),
    Box([Label(value='Adyacentes Elemento')], layout=form_item_layout),
    Box([listaInputNode], layout=form_item_layout),
    Box([btn_findAdjNodeArbolOption], layout=form_item_layout),
]

items_optionsFindAllNodesArbol = [
    Box([Label(value=' ')]),
    Box([Label(value='Listar todos los nodos')], layout=form_item_layout),
    Box([Label(value='Orden:')]),
    Box([listaOrden]),
    Box([btn_todosNodosArbol]),
]

layoutOptionsCreateArbol = Layout(
    display='none',
    flex_flow='column',
    align_items='stretch',
    width='auto',
    justify_content='center'
)
layoutOptionsAddNodeArbol = Layout(
    display='none',
    flex_flow='column',
    align_items='stretch',
    width='auto',
    justify_content='center'
)
layoutOptionsDeleteNodeArbol = Layout(
    display='none',
    flex_flow='column',
    align_items='stretch',
    width='auto',
    justify_content='center'
)
layoutOptionsFindNodeArbol = Layout(
    display='none',
    flex_flow='column',
    align_items='stretch',
    width='auto',
    justify_content='center'
)
layoutOptionsFindAdjNodeArbol = Layout(
    display='none',
    flex_flow='column',
    align_items='stretch',
    width='auto',
    justify_content='center'
)
layoutOptionsFindAllNodesArbol = Layout(
    display='none',
    flex_flow='column',
    align_items='stretch',
    width='auto',
    justify_content='center'
)

optionsCreateArbol = Box(items_optionsCreateArbol, layout=layoutOptionsCreateArbol)
optionsAddNodeArbol = Box(items_optionsAddNodeArbol, layout=layoutOptionsAddNodeArbol)
optionsDeleteNodeArbol = Box(items_optionsDeleteNodeArbol, layout=layoutOptionsDeleteNodeArbol)
optionsFindNodeArbol = Box(items_optionsFindNodeArbol, layout=layoutOptionsFindNodeArbol)
optionsFindAdjNodeArbol = Box(items_optionsFindAdjNodeArbol, layout=layoutOptionsFindAdjNodeArbol)
optionsFindAllNodesArbol = Box(items_optionsFindAllNodesArbol, layout=layoutOptionsFindAllNodesArbol)

#### Layout - Grafos

In [20]:
items_optionsCreateGrafo = [
    Box([Label(value='Crear Grafo')], layout=form_item_layout),
    Box([grafoLabels]),
    Box([Label(value='Tipo:')]),
    Box([grafoType]),
    Box([Label(value='Inicialización:')]),
    Box([listaInit]),
    Box([btn_crearGrafoOption]),
]
items_optionsAddNodeGrafo = [
    Box([Label(value=' ')]),
    Box([Label(value='Añadir Nodo')], layout=form_item_layout),
    Box([grafoLabels]),
    Box([listaInputNode], layout=form_item_layout),
    Box([btn_addNodeGrafoOption], layout=form_item_layout),
]
items_optionsDeleteNodeGrafo = [
    Box([Label(value=' ')]),
    Box([Label(value='Eliminar Nodo')], layout=form_item_layout),
    Box([grafoLabels]),
    Box([listaInputNode], layout=form_item_layout),
    Box([btn_deleteNodeGrafoOption], layout=form_item_layout),
]
items_optionsFindNodeGrafo = [
    Box([Label(value=' ')]),
    Box([Label(value='Existe Nodo')], layout=form_item_layout),
    Box([grafoLabels]),
    Box([listaInputNode], layout=form_item_layout),
    Box([btn_findNodeGrafoOption], layout=form_item_layout),
]
items_optionsFindAdjNodeGrafo = [
    Box([Label(value=' ')]),
    Box([Label(value='Adyacentes Elemento')], layout=form_item_layout),
    Box([grafoLabels]),
    Box([listaInputNode], layout=form_item_layout),
    Box([btn_findAdjNodeGrafoOption], layout=form_item_layout),
]
items_optionsAddEdgeGrafo = [
    Box([Label(value=' ')]),
    Box([Label(value='Añadir Arco')], layout=form_item_layout),
    Box([grafoLabels]),
    Box([grafoInputNodeInit], layout=form_item_layout),
    Box([grafoInputNodeEnd], layout=form_item_layout),
    Box([grafoInputEdgeWeight], layout=form_item_layout),
    Box([btn_addEdgeOption], layout=form_item_layout),
]
items_optionsAllNodesGrafo = [
    Box([Label(value=' ')]),
    Box([grafoLabels]),
    Box([btn_todosNodosGrafo], layout=form_item_layout),
]
items_optionsRecorridosGrafo = [
    Box([grafoLabels]),
    Box([Label(value='Recorridos Grafo')], layout=form_item_layout),
    Box([grafoInputNode], layout=form_item_layout),
    Box([grafoRecorrido], layout=form_item_layout),
    Box([btn_recorridosGrafo], layout=form_item_layout),
]

layoutOptionsRecorridosGrafo = Layout(
    display='none',
    flex_flow='column',
    align_items='stretch',
    width='auto',
    justify_content='center'
)
layoutOptionsAllNodesGrafo = Layout(
    display='none',
    flex_flow='column',
    align_items='stretch',
    width='auto',
    justify_content='center'
)
layoutOptionsCreateGrafo = Layout(
    display='none',
    flex_flow='column',
    align_items='stretch',
    width='auto',
    justify_content='center'
)
layoutOptionsAddNodeGrafo = Layout(
    display='none',
    flex_flow='column',
    align_items='stretch',
    width='auto',
    justify_content='center'
)
layoutOptionsDeleteNodeGrafo = Layout(
    display='none',
    flex_flow='column',
    align_items='stretch',
    width='auto',
    justify_content='center'
)
layoutOptionsFindNodeGrafo = Layout(
    display='none',
    flex_flow='column',
    align_items='stretch',
    width='auto',
    justify_content='center'
)
layoutOptionsAddEdgeGrafo = Layout(
    display='none',
    flex_flow='column',
    align_items='stretch',
    width='auto',
    justify_content='center'
)
layoutOptionsFindAdjNodeGrafo = Layout(
    display='none',
    flex_flow='column',
    align_items='stretch',
    width='auto',
    justify_content='center'
)

optionsCreateGrafo = Box(items_optionsCreateGrafo, layout=layoutOptionsCreateGrafo)
optionsAddNodeGrafo = Box(items_optionsAddNodeGrafo, layout=layoutOptionsAddNodeGrafo)
optionsDeleteNodeGrafo = Box(items_optionsDeleteNodeGrafo, layout=layoutOptionsDeleteNodeGrafo)
optionsFindNodeGrafo = Box(items_optionsFindNodeGrafo, layout=layoutOptionsFindNodeGrafo)
optionsFindAdjNodeGrafo = Box(items_optionsFindAdjNodeGrafo, layout=layoutOptionsFindAdjNodeGrafo)
optionsAddEdgeGrafo = Box(items_optionsAddEdgeGrafo, layout=layoutOptionsAddEdgeGrafo)
optionsAllNodesGrafo = Box(items_optionsAllNodesGrafo, layout=layoutOptionsAllNodesGrafo)
optionsRecorridosGrafo = Box(items_optionsRecorridosGrafo, layout=layoutOptionsRecorridosGrafo)

#### Layout - Nueva Estructura

In [21]:
# -------------------------- Cargar Estructura - Nueva --------------------------#
#layoutOptionsNuevoBoton = Layout(
#    display='none',
#    flex_flow='column',
#    align_items='stretch',
#    width='auto',
#    justify_content='center'
#)
#
#items_optionsNuevoBoton = [
#    # Insertar los elementos que se desean para el detalle de la nueva opcion
#    Box([Label(value='Nuevo Boton')], layout=form_item_layout),
#    Box([btn_ejecutarNuevaOpcion], layout=form_item_layout),
#]
#
#optionsNuevoBoton = Box(items_optionsNuevoBoton, layout=layoutOptionsNuevoBoton)
#
# ------------------------------------------------------------------------------#

### Contruir Interfaz

En esta sección se unen todos los elementos de la interfaz para posteriormente crearla

#### Contruir Layouts

In [22]:
allOptions = [optionsCreateList,
            optionsAddNode,
            optionsDeleteNode,
            optionsFindNode,
            optionsFindAdjNode,
            optionsFindAllNodes,
            optionsCreateArbol,
            optionsAddNodeArbol,
            optionsDeleteNodeArbol,
            optionsFindNodeArbol,
            optionsFindAdjNodeArbol,
            optionsFindAllNodesArbol,
            optionsCreateGrafo,
            optionsAddNodeGrafo,
            optionsDeleteNodeGrafo,
            optionsFindNodeGrafo,
            optionsAddEdgeGrafo,
            optionsFindAdjNodeGrafo,
            optionsAllNodesGrafo,
            optionsRecorridosGrafo,
            # -------------------------- Cargar Estructura - Nueva --------------------------#
            #optionsNuevoBoton
            # ------------------------------------------------------------------------------#
            ]            

def hideShowOptions(option):
    for i in allOptions:
        i.layout.visibility = 'hidden'
        i.layout.display = 'none'
    listaInputNode.value = ''
    grafoInputNodeInit.value = ''
    grafoInputNodeEnd.value = ''
    grafoInputEdgeWeight.value = 0.00
       
    option.layout.display = 'block'
    option.layout.visibility = "visible"

# -----------------------------Panel de Opciones Lista Enlazada-----------------------------
btn_crearLista = Button(description='Crear Lista Enlazada', width='extended')
def on_button_crearLista_clicked(b):
    with out:
        hideShowOptions(optionsCreateList)
btn_crearLista.on_click(on_button_crearLista_clicked)

btn_addNode = Button(description='Añadir Elemento', width='extended')
def on_button_addNode_clicked(b):
    with out:
        hideShowOptions(optionsAddNode)
btn_addNode.on_click(on_button_addNode_clicked)

btn_deleteNode = Button(description='Eliminar Elemento', width='extended')
def on_button_deleteNode_clicked(b):
    with out:
        hideShowOptions(optionsDeleteNode)
btn_deleteNode.on_click(on_button_deleteNode_clicked)

btn_findNode = Button(description='Existe Elemento', width='extended')
def on_button_findNode_clicked(b):
    with out:
        hideShowOptions(optionsFindNode)
btn_findNode.on_click(on_button_findNode_clicked)

btn_findAdjNode = Button(description='Encontrar Adyacentes', width='extended')
def on_button_findAdjNode_clicked(b):
    with out:
        hideShowOptions(optionsFindAdjNode)
btn_findAdjNode.on_click(on_button_findAdjNode_clicked)

btn_findAllNodes = Button(description='Encontrar Todos', width='extended')
def on_button_findAllNodes_clicked(b):
    with out:
        hideShowOptions(optionsFindAllNodes)
btn_findAllNodes.on_click(on_button_findAllNodes_clicked)

# -----------------------------Panel de Opciones Arbol-----------------------------
btn_crearArbol = Button(description='Crear Árbol Binario', width='extended')
def on_button_crearArbol_clicked(b):
    with out:
        hideShowOptions(optionsCreateArbol)
btn_crearArbol.on_click(on_button_crearArbol_clicked)

btn_addNodeArbol = Button(description='Añadir Elemento', width='extended')
def on_button_addNodeArbol_clicked(b):
    with out:
        hideShowOptions(optionsAddNodeArbol)
btn_addNodeArbol.on_click(on_button_addNodeArbol_clicked)

btn_deleteNodeArbol = Button(description='Eliminar Elemento', width='extended')
def on_button_deleteNodeArbol_clicked(b):
    with out:
        hideShowOptions(optionsDeleteNodeArbol)
btn_deleteNodeArbol.on_click(on_button_deleteNodeArbol_clicked)

btn_findNodeArbol = Button(description='Existe Elemento', width='extended')
def on_button_findNodeArbol_clicked(b):
    with out:
        hideShowOptions(optionsFindNodeArbol)
btn_findNodeArbol.on_click(on_button_findNodeArbol_clicked)

btn_findAdjNodeArbol = Button(description='Encontrar Adyacentes', width='extended')
def on_button_findAdjNodeArbol_clicked(b):
    with out:
        hideShowOptions(optionsFindAdjNodeArbol)
btn_findAdjNodeArbol.on_click(on_button_findAdjNodeArbol_clicked)

btn_findAllNodesArbol = Button(description='Encontrar Todos', width='extended')
def on_button_findAllNodesArbol_clicked(b):
    with out:
        hideShowOptions(optionsFindAllNodesArbol)
btn_findAllNodesArbol.on_click(on_button_findAllNodesArbol_clicked)

# -----------------------------Panel de Opciones Grafo-----------------------------
btn_crearGrafo = Button(description='Crear Grafo', width='extended')
def on_button_crearGrafo_clicked(b):
    with out:
        hideShowOptions(optionsCreateGrafo)
btn_crearGrafo.on_click(on_button_crearGrafo_clicked)

btn_addNodeGrafo = Button(description='Añadir Nodo', width='extended')
def on_button_addNodeGrafo_clicked(b):
    with out:
        hideShowOptions(optionsAddNodeGrafo)
btn_addNodeGrafo.on_click(on_button_addNodeGrafo_clicked)

btn_deleteNodeGrafo = Button(description='Eliminar Nodo', width='extended')
def on_button_deleteNodeGrafo_clicked(b):
    with out:
        hideShowOptions(optionsDeleteNodeGrafo)
btn_deleteNodeGrafo.on_click(on_button_deleteNodeGrafo_clicked)

btn_findNodeGrafo = Button(description='Existe Nodo', width='extended')
def on_button_findNodeGrafo_clicked(b):
    with out:
        hideShowOptions(optionsFindNodeGrafo)
btn_findNodeGrafo.on_click(on_button_findNodeGrafo_clicked)

btn_addEdgeGrafo = Button(description='Añadir Arco', width='extended')
def on_button_addEdgeGrafo_clicked(b):
    with out:
        hideShowOptions(optionsAddEdgeGrafo)
btn_addEdgeGrafo.on_click(on_button_addEdgeGrafo_clicked)

btn_findAdjNodeGrafo = Button(description='Encontrar Adyacentes', width='extended')
def on_button_findAdjNodeGrafo_clicked(b):
    with out:
        hideShowOptions(optionsFindAdjNodeGrafo)
btn_findAdjNodeGrafo.on_click(on_button_findAdjNodeGrafo_clicked)

btn_findAllNodesGrafo = Button(description='Encontrar Todos', width='extended')
def on_button_findAllNodesGrafo_clicked(b):
    with out:
        hideShowOptions(optionsAllNodesGrafo)
btn_findAllNodesGrafo.on_click(on_button_findAllNodesGrafo_clicked)

btn_recorridosOptionGrafo = Button(description='Realizar Recorrido', width='extended')
def on_button_recorridosOptionsGrafo_clicked(b):
    with out:
        hideShowOptions(optionsRecorridosGrafo)
btn_recorridosOptionGrafo.on_click(on_button_recorridosOptionsGrafo_clicked)

# -------------------------- Cargar Estructura - Nueva --------------------------#
#btn_nuevoBoton = Button(description='Nuevo Boton', width='extended')
#def on_button_nuevoBoton_clicked(b):
#    with out:
#        hideShowOptions(optionsNuevoBoton)
#btn_nuevoBoton.on_click(on_button_nuevoBoton_clicked)
# ------------------------------------------------------------------------------#

#### Construir Opciones

In [23]:
layout_canvas = Layout(height='590px', border='1px dotted blue', overflow ='auto')
layout_options = Layout(height='auto', width='auto')

buttons_listas = [
    Box([btn_crearLista], layout=form_item_layout),
    Box([btn_addNode], layout=form_item_layout),
    Box([btn_deleteNode], layout=form_item_layout),
    Box([btn_findNode], layout=form_item_layout),
    Box([btn_findAdjNode], layout=form_item_layout),
    Box([btn_findAllNodes], layout=form_item_layout),
]
buttons_arboles = [
    Box([btn_crearArbol], layout=form_item_layout),
    Box([btn_addNodeArbol], layout=form_item_layout),
    Box([btn_deleteNodeArbol], layout=form_item_layout),
    Box([btn_findNodeArbol], layout=form_item_layout),
    Box([btn_findAdjNodeArbol], layout=form_item_layout),
    Box([btn_findAllNodesArbol],layout=form_item_layout)
]
buttons_grafos = [
    Box([btn_crearGrafo], layout=form_item_layout),
    Box([btn_addNodeGrafo], layout=form_item_layout),
    Box([btn_deleteNodeGrafo], layout=form_item_layout),
    Box([btn_findNodeGrafo], layout=form_item_layout),
    Box([btn_addEdgeGrafo], layout=form_item_layout),
    Box([btn_findAdjNodeGrafo], layout=form_item_layout),
    Box([btn_findAllNodesGrafo], layout=form_item_layout),
    Box([btn_recorridosOptionGrafo], layout=form_item_layout),
]

# Cargar Estructura - Lista Enlazada
btn_cargarEstructuraLista = Button(description='Cargar Lista', width='extended', button_style='warning')
def on_button_cargarEstructuraLista_clicked(b):
    with out:       
        try:
            root = tk.Tk()
            src_path = tk.filedialog.askopenfilename(title='Select a file...',filetypes=[("Python File", "*.py")])
            root.destroy()
            spec = importlib.util.spec_from_file_location("",src_path)
            foo = importlib.util.module_from_spec(spec)
  
            spec.loader.exec_module(foo)
            this.file = foo
            out.clear_output()
            print('Estructura Cargada Exitosamente')
            print(src_path)
        except:
            out.clear_output
            if src_path == '':
                e = "Problema al cargar la estructura: No se seleccionó ningún archivo"
            else:
                e = "Problema al cargar la estructura: " + src_path
            print(e)
        buttonsListas.layout.visibility = 'visible'
        buttonsListas.layout.display = 'block'
btn_cargarEstructuraLista.on_click(on_button_cargarEstructuraLista_clicked)

buttons_cargarEstructurasLista = [
    Box([btn_cargarEstructuraLista], layout=form_item_layout),
]

# Cargar Estructura - Arbol BST
btn_cargarEstructuraBST = Button(description='Cargar BST', width='extended', button_style='warning')
def on_button_cargarEstructuraBST_clicked(b):
    with out:       
        try:
            root = tk.Tk()
            src_path = tk.filedialog.askopenfilename(title='Select a file...',filetypes=[("Python File", "*.py")])
            root.destroy()
            spec = importlib.util.spec_from_file_location("",src_path)
            foo = importlib.util.module_from_spec(spec)
  
            spec.loader.exec_module(foo)
            this.file = foo
            out.clear_output()
            print('Estructura Cargada Exitosamente')
            print(src_path)
        except:
            out.clear_output
            if src_path == '':
                e = "Problema al cargar la estructura: No se seleccionó ningún archivo"
            else:
                e = "Problema al cargar la estructura: " + src_path
            print(e)
        buttonsArboles.layout.visibility = 'visible'
        buttonsArboles.layout.display = 'block'
btn_cargarEstructuraBST.on_click(on_button_cargarEstructuraBST_clicked)

buttons_cargarEstructurasBST = [
    Box([btn_cargarEstructuraBST], layout=form_item_layout),
]

# Cargar Estructura - Grafo
btn_cargarEstructuraGrafo = Button(description='Cargar Grafo', width='extended', button_style='warning')
def on_button_cargarEstructuraGrafo_clicked(b):
    with out:       
        try:
            root = tk.Tk()
            src_path = tk.filedialog.askopenfilename(title='Select a file...',filetypes=[("Python File", "*.py")])
            root.destroy()
            spec = importlib.util.spec_from_file_location("",src_path)
            foo = importlib.util.module_from_spec(spec)
  
            spec.loader.exec_module(foo)
            this.file = foo
            out.clear_output()
            print('Estructura Cargada Exitosamente')
            print(src_path)
        except:
            out.clear_output
            if src_path == '':
                e = "Problema al cargar la estructura: No se seleccionó ningún archivo"
            else:
                e = "Problema al cargar la estructura: " + src_path
            print(e)
        buttonsGrafos.layout.visibility = 'visible'
        buttonsGrafos.layout.display = 'block'
btn_cargarEstructuraGrafo.on_click(on_button_cargarEstructuraGrafo_clicked)

buttons_cargarEstructurasGrafo = [
    Box([btn_cargarEstructuraGrafo], layout=form_item_layout),
]


buttonsCargarLista = Box(buttons_cargarEstructurasLista, layout=Layout(
    display='flex',
    flex_flow='column',
    align_items='stretch',
    width='auto',
    justify_content='center'
))
buttonsCargarBST = Box(buttons_cargarEstructurasBST, layout=Layout(
    display='flex',
    flex_flow='column',
    align_items='stretch',
    width='auto',
    justify_content='center'
))

buttonsCargarGrafo = Box(buttons_cargarEstructurasGrafo, layout=Layout(
    display='flex',
    flex_flow='column',
    align_items='stretch',
    width='auto',
    justify_content='center'
))

buttonsListas = Box(buttons_listas, layout=Layout(
    display='none',
    flex_flow='column',
    align_items='stretch',
    width='auto',
    justify_content='center'
))
buttonsArboles = Box(buttons_arboles, layout=Layout(
    display='none',
    flex_flow='column',
    align_items='stretch',
    width='auto',
    justify_content='center'
))
buttonsGrafos = Box(buttons_grafos, layout=Layout(
    display='none',
    flex_flow='column',
    align_items='stretch',
    width='auto',
    justify_content='center'
))

canvas = HBox([out], layout=layout_canvas)
optionsListas = VBox([buttonsCargarLista,
                      buttonsListas,
                      optionsCreateList,
                      optionsAddNode,
                      optionsDeleteNode,
                      optionsFindNode,
                      optionsFindAdjNode,
                      optionsFindAllNodes],
                      layout=layout_options)
optionsArboles = VBox([buttonsCargarBST,
                       buttonsArboles,
                        optionsCreateArbol,
                        optionsAddNodeArbol,
                        optionsDeleteNodeArbol,
                        optionsFindNodeArbol,
                        optionsFindAdjNodeArbol,
                        optionsFindAllNodesArbol],
                        layout=layout_options)
optionsGrafos = VBox([buttonsCargarGrafo,
                    buttonsGrafos,            
                    optionsCreateGrafo,
                    optionsAddNodeGrafo,
                    optionsDeleteNodeGrafo,
                    optionsFindNodeGrafo,
                    optionsAddEdgeGrafo,
                    optionsFindAdjNodeGrafo,
                    optionsAllNodesGrafo,
                    optionsRecorridosGrafo],
                    layout=layout_options)

# -------------------------- Cargar Estructura - Nueva --------------------------#
#btn_cargarNueva = Button(description='Cargar Nueva', width='extended', button_style='warning')
#def on_button_cargarNueva_clicked(b):
#    with out:       
#        try:
#            root = tk.Tk()
#            src_path = tk.filedialog.askopenfilename(title='Select a file...',filetypes=[("Python File", "*.py")])
#            root.destroy()
#            spec = importlib.util.spec_from_file_location("",src_path)
#            foo = importlib.util.module_from_spec(spec)
#  
#            spec.loader.exec_module(foo)
#            this.file = foo
#            out.clear_output()
#            print('Estructura Cargada Exitosamente')
#            print(src_path)
#        except:
#            out.clear_output
#            if src_path == '':
#                e = "Problema al cargar la estructura: No se seleccionó ningún archivo"
#            else:
#                e = "Problema al cargar la estructura: " + src_path
#            print(e)
#        buttonsNueva.layout.visibility = 'visible'
#        buttonsNueva.layout.display = 'block'
#btn_cargarNueva.on_click(on_button_cargarNueva_clicked)
#
#
#buttons_nueva = [
#    Box([btn_nuevoBoton], layout=form_item_layout),
#]
#
#buttons_cargarEstructurasNueva = [
#    Box([btn_cargarNueva], layout=form_item_layout),
#]
#
#buttonsCargarNuevaEstructura = Box(buttons_cargarEstructurasNueva, layout=Layout(
#    display='flex',
#    flex_flow='column',
#    align_items='stretch',
#    width='auto',
#    justify_content='center'
#))
#
#buttonsNueva = Box(buttons_nueva, layout=Layout(
#    display='none',
#    flex_flow='column',
#    align_items='stretch',
#    width='auto',
#    justify_content='center'
#))
#
#optionsNuevaEstructura = VBox([buttonsCargarNuevaEstructura,
#                    buttonsNueva,            
#                    optionsNuevoBoton],
#                    layout=layout_options)
# ------------------------------------------------------------------------------#

#### Construir Aplicación

In [24]:
with out1:
    app = AppLayout(header=None,
          left_sidebar=None,
          center=canvas,
          right_sidebar=optionsListas,
          pane_widths=['0%', '80%', '20%'],
          pane_heights=['0%','590px', 1],
          footer=None)
    display(app)
with out2:
    app = AppLayout(header=None,
          left_sidebar=None,
          center=canvas,
          right_sidebar=optionsArboles,
          pane_widths=['0%', '80%', '20%'],
          pane_heights=['0%','590px', 1],
          footer=None)
    display(app)
with out3:
    app = AppLayout(header=None,
          left_sidebar=None,
          center=canvas,
          right_sidebar=optionsGrafos,
          pane_widths=['0%', '80%', '20%'],
          pane_heights=['0%','590px', 1],
          footer=None)
    display(app)

## Mostrar Aplicación

In [25]:
display(tab) 